In [1]:
import ast

In [2]:
def calc_change(
    parsed_vars,
    top_change,
    num_pers,
    life_val,
    waiver,
    prop_val,
    opt_num,
    change_num,
    person_num,
):
    top_change_wt = 0.0
    rights_change_wt = 0.0
    if top_change == 1:
        # 1 - Increasing/decreasing existential risks
        # n_die1 = Total people expected to die
        n_die1 = parsed_vars.get("n_die1")[opt_num][change_num]
        life_val = life_value_fn(parsed_vars, opt_num, change_num)
        # n_pain1 = Total people expected to experience pain
        n_pain1 = parsed_vars.get("n_pain1")[opt_num][change_num]
        pain = unwanted_pain_fn(parsed_vars, opt_num, change_num)
        # na_die1 = How many animals expected to die
        na_die1 = parsed_vars.get("na_die1")[opt_num][change_num]
        alife_val = animal_dying_fn(parsed_vars, opt_num, change_num)
        # na_pain1 = How many animals expected to experience pain
        na_pain1 = parsed_vars.get("na_pain1")[opt_num][change_num]
        apain = animal_pain_fn(parsed_vars, opt_num, change_num)
        # np_die1 = How many plants expected to die
        np_die1 = parsed_vars.get("np_die1")[opt_num][change_num]
        plife_val = plant_dying_fn(parsed_vars, opt_num, change_num)
        # n_notborn1 = How many people estimated to never get chance to be born
        n_notborn1 = parsed_vars.get("n_notborn1")[opt_num][change_num]
        # na_notborn1 = How many animals estimated to never get chance to be born
        na_notborn1 = parsed_vars.get("na_notborn1")[opt_num][change_num]
        # np_notborn1 = How many plants estimated to never get chance to live
        np_notborn1 = parsed_vars.get("np_notborn1")[opt_num][change_num]
        # time_prob1 = Time period over which the probability change is expected to hold (in years)
        time_prob1 = parsed_vars.get("time_prob1")[opt_num][change_num]
        # prob_i1 = Starting probability (in percent)
        prob_i1 = parsed_vars.get("prob_i1")[opt_num][change_num]
        # prob_f1 = Ending probability (in percent)
        prob_f1 = parsed_vars.get("prob_f1")[opt_num][change_num]
        # Assume people are in 0% harm's way and have 0% culpability for extinction
        top_change_wt = (
            (prob_i1 - prob_f1)
            / 100
            * (
                n_die1 * (life_val + 1000000.0)
                + n_pain1 * (pain * 1000000.0 + pain)
                + na_die1 * alife_val
                + na_pain1 * apain
                + (np_die1 + np_notborn1) * plife_val
                + n_notborn1 * 80 * 0.6 / 54.0
                + na_notborn1 * 3 * 0.6 / 540.0
            )
        )
        rights_change_wt = (
            (prob_i1 - prob_f1)
            / 100
            * (n_die1 * (1000000.0) + n_pain1 * (pain * 1000000.0))
        )
    elif top_change == 2:
        # 2 - Someone dying, or increasing/decreasing its probability
        # *****Note that expected utility is a placeholder for dealing with probabilities, and will likely be modified in the future
        # time_prob2 = Time period over which the probability change is expected to hold (in years)
        time_prob2 = parsed_vars.get("time_prob2")[opt_num][change_num]
        # prob_i2 = Starting probability (in percent)
        prob_i2 = parsed_vars.get("prob_i2")[opt_num][change_num]
        # prob_f2 = Ending probability (in percent)
        prob_f2 = parsed_vars.get("prob_f2")[opt_num][change_num]
        villainy = 0
        if num_pers == 0:
            # n_die2 = How many effectively equivalent people would die
            n_die2 = parsed_vars.get("n_die2")[opt_num][change_num]
            # waiver2 = Percentage of involved people who'd likely want to waive consideration of this value
            # destruction for them if they knew of a bigger corresponding value destruction
            waiver2 = parsed_vars.get("waiver2")[opt_num][change_num]
            # defaults = Do you want to use default values for this person's life value (Y or N)?
            defaults = parsed_vars.get("defaults")
            if defaults == "Y":
                change = 1.54586
            else:
                change = life_value_fn(parsed_vars, opt_num, change_num)
        else:
            n_die2 = num_pers
            waiver2 = waiver
            change = life_val
            # villainy = Did this person put anyone in harm’s way (0 for no, 1 for yes)?
            villainy = parsed_vars.get("villainy")[person_num - 1]
        if villainy == 1:
            villain_fact = 0.0001
        else:
            villain_fact = 1
        top_change_wt = (
            villain_fact
            * (prob_i2 - prob_f2)
            / 100
            * n_die2
            * change
            * (1.0 - waiver2 / 100.0)
        )
    elif top_change == 3:
        # 3 - Non-freely chosen physical pain for a person, or increasing/decreasing its probability
        time_prob3 = parsed_vars.get("time_prob3")[opt_num][change_num]
        prob_i3 = parsed_vars.get("prob_i3")[opt_num][change_num]
        prob_f3 = parsed_vars.get("prob_f3")[opt_num][change_num]
        if num_pers == 0:
            # n_pain3 = How many effectively equivalent people would experience pain
            n_pain3 = parsed_vars.get("n_pain3")[opt_num][change_num]
            waiver3 = parsed_vars.get("waiver3")[opt_num][change_num]
        else:
            n_pain3 = num_pers
            waiver3 = waiver
        change = unwanted_pain_fn(parsed_vars, opt_num, change_num)
        top_change_wt = (
            n_pain3 * (prob_i3 - prob_f3) / 100 * change * (1.0 - waiver3 / 100.0)
        )
    elif top_change == 4:
        # 4 - Loss of function for a human, or increasing/decreasing its probability
        time_prob4 = parsed_vars.get("time_prob4")[opt_num][change_num]
        prob_i4 = parsed_vars.get("prob_i4")[opt_num][change_num]
        prob_f4 = parsed_vars.get("prob_f4")[opt_num][change_num]
        if num_pers == 0:
            # n_health4 = How many effectively equivalent people would have loss of function
            n_health4 = parsed_vars.get("n_health4")[opt_num][change_num]
            waiver4 = parsed_vars.get("waiver4")[opt_num][change_num]
        else:
            n_health4 = num_pers
            waiver4 = waiver
        change = function_loss_fn(parsed_vars, opt_num, change_num)
        top_change_wt = (
            n_health4 * (prob_i4 - prob_f4) / 100 * change * (1.0 - waiver4 / 100.0)
        )
    elif top_change == 5:
        # 5 - Bringing life into the world with insufficient resources/lack of intent to support it or increasing/decreasing the probability of this
        time_prob5 = parsed_vars.get("time_prob5")[opt_num][change_num]
        prob_i5 = parsed_vars.get("prob_i5")[opt_num][change_num]
        prob_f5 = parsed_vars.get("prob_f5")[opt_num][change_num]
        # For bringing human life into world with insufficient resources/lack of intent to support it:
        top_change_wth = bring_human_notenough_fn(parsed_vars, opt_num, change_num)
        # For bring animal life into world with insufficient resources/lack of intent to support it:
        top_change_wta = bring_animal_notenough_fn(parsed_vars, opt_num, change_num)
        top_change_wt = (top_change_wth + top_change_wta) * (prob_i5 - prob_f5) / 100
    elif top_change == 6:
        # 6 - Bringing life into the world with sufficient resources/intent to support it or decreasing/increasing the probability of this
        time_prob6 = parsed_vars.get("time_prob6")[opt_num][change_num]
        prob_i6 = parsed_vars.get("prob_i6")[opt_num][change_num]
        prob_f6 = parsed_vars.get("prob_f6")[opt_num][change_num]
        # n_hum6 = How many humans brought into world
        n_hum6 = parsed_vars.get("n_hum6")[opt_num][change_num]
        # qol6 = Average quality of life expected (0 to 1)
        qol6 = parsed_vars.get("qol6")[opt_num][change_num]
        # For bring animal life into world with insufficient resources/lack of intent to support it:
        # n_anim6 = How many animals brought into world
        n_anim6 = parsed_vars.get("n_anim6")[opt_num][change_num]
        # qol_anim6 = Average quality of life expected (0 to 1)
        qol_anim6 = parsed_vars.get("qol_anim6")[opt_num][change_num]
        # anim_value6 = How valued the animal is to humans
        anim_value6 = parsed_vars.get("anim_value6")[opt_num][change_num]
        # For bringing more plant life into the world:
        # n_plant6 = How many non-tree plants brought into world
        n_plant6 = parsed_vars.get("n_plant6")[opt_num][change_num]
        # plant_value6 = How valued the plant is to humans in terms of beauty
        plant_value6 = parsed_vars.get("plant_value6")[opt_num][change_num]
        # n_tree6 = How many trees brought into world
        n_tree6 = parsed_vars.get("n_tree6")[opt_num][change_num]
        # tree_value6 = How valued the tree is to humans in terms of beauty
        tree_value6 = parsed_vars.get("tree_value6")[opt_num][change_num]
        # Assume non-tree plant lives 30 years, tree lives 150 years
        top_change_wt = (
            (prob_i6 - prob_f6)
            / 100
            * (
                n_hum6 * qol6 * 78 / 54
                + n_anim6 * (qol_anim6 * 3 / 540 + anim_value6)
                + n_plant6 * (plant_value6 + 30 / 540000)
                + n_tree6 * (tree_value6 + 150 / 54000)
            )
        )
    elif top_change == 7:
        # 7 - Extinction of animal or plant species or increasing/decreasing its probability
        time_prob7 = parsed_vars.get("time_prob7")[opt_num][change_num]
        prob_i7 = parsed_vars.get("prob_i7")[opt_num][change_num]
        prob_f7 = parsed_vars.get("prob_f7")[opt_num][change_num]
        # wt_loss_hum7 = Weight of loss to humans (emotional, financial, learning)
        wt_loss_hum7 = parsed_vars.get("wt_loss_hum7")[opt_num][change_num]
        # wt_loss_eco7 = Weight of loss to ecosystem (animals dying, other species going extinct,
        # draught, flood protection decrease)
        wt_loss_eco7 = parsed_vars.get("wt_loss_eco7")[opt_num][change_num]
        top_change_wt = (wt_loss_hum7 + wt_loss_eco7) * (prob_i7 - prob_f7) / 100
    elif top_change == 8:
        # 8 - Threat (by someone) of physical violence or emotional pain, or increasing/decreasing its probability
        time_prob8 = parsed_vars.get("time_prob8")[opt_num][change_num]
        prob_i8 = parsed_vars.get("prob_i8")[opt_num][change_num]
        prob_f8 = parsed_vars.get("prob_f8")[opt_num][change_num]
        tpain = threat_pain_fn(parsed_vars, opt_num, change_num)
        thealth = threat_function_loss_fn(parsed_vars, opt_num, change_num)
        tkill = threat_kill_fn(parsed_vars, opt_num, change_num)
        top_change_wt = (tpain + thealth + tkill) * (prob_i8 - prob_f8) / 100
    elif top_change == 9:
        # 9 - Emotional abuse of a child or increasing/decreasing its probability
        time_prob9 = parsed_vars.get("time_prob9")[opt_num][change_num]
        prob_i9 = parsed_vars.get("prob_i9")[opt_num][change_num]
        prob_f9 = parsed_vars.get("prob_f9")[opt_num][change_num]
        # years_remain9 = Expected years of life remaining
        years_remain9 = parsed_vars.get("years_remain9")[opt_num][change_num]
        # qol_decrease9 = Average quality of life decrease due to abuse (0 to 2)
        qol_decrease9 = parsed_vars.get("qol_decrease9")[opt_num][change_num]
        top_change_wt = years_remain9 * qol_decrease9 / 54.0 * (prob_i9 - prob_f9) / 100
    elif top_change == 10:
        # 10 - Emotional pain or increasing/decreasing its probability
        time_prob10 = parsed_vars.get("time_prob10")[opt_num][change_num]
        prob_i10 = parsed_vars.get("prob_i10")[opt_num][change_num]
        prob_f10 = parsed_vars.get("prob_f10")[opt_num][change_num]
        top_change_wt = (
            emotional_pain_fn(parsed_vars, opt_num, change_num)
            * (prob_i10 - prob_f10)
            / 100
        )
    elif top_change == 11:
        # 11 - Words or actions that needlessly hurt someone’s reputation, or increasing/decreasing their probabilities
        time_prob11 = parsed_vars.get("time_prob11")[opt_num][change_num]
        prob_i11 = parsed_vars.get("prob_i11")[opt_num][change_num]
        prob_f11 = parsed_vars.get("prob_f11")[opt_num][change_num]
        # salary_decrease11 = Decrease in salary ($/year)
        salary_decrease11 = parsed_vars.get("salary_decrease11")[opt_num][change_num]
        # years11 = Expected duration of hurt reputation (years)
        years11 = parsed_vars.get("years11")[opt_num][change_num]
        # n_interact11 = Average number of people per year interact with
        n_interact11 = parsed_vars.get("n_interact11")[opt_num][change_num]
        # value_add11 = Average value add decrease per person
        value_add11 = parsed_vars.get("value_add11")[opt_num][change_num]
        top_change_wt = (
            (
                salary_decrease11 / 4050000 * years11
                + years11 * n_interact11
                + value_add11
            )
            * (prob_i11 - prob_f11)
            / 100
        )
    elif top_change == 12:
        # 12 - Words or actions that deservedly improve someone’s reputation, or decreasing/increasing their probabilities
        time_prob12 = parsed_vars.get("time_prob12")[opt_num][change_num]
        prob_i12 = parsed_vars.get("prob_i12")[opt_num][change_num]
        prob_f12 = parsed_vars.get("prob_f12")[opt_num][change_num]
        # income_up12 = Increase in income ($/year)
        income_up12 = parsed_vars.get("income_up12")[opt_num][change_num]
        # years12 = Expected duration of improved reputation (years)
        years12 = parsed_vars.get("years12")[opt_num][change_num]
        # n_interact12 = Average number of people per year interact with
        n_interact12 = parsed_vars.get("n_interact12")[opt_num][change_num]
        # value_add12 = Average value add increase per person
        value_add12 = parsed_vars.get("value_add12")[opt_num][change_num]
        top_change_wt = (
            (income_up12 / 4050000 * years12 + years12 * n_interact12 + value_add12)
            * (prob_i12 - prob_f12)
            / 100
        )
    elif top_change == 13:
        # 13 - Damaging/destroying/defacing property or increasing/decreasing its probability
        time_prob13 = parsed_vars.get("time_prob13")[opt_num][change_num]
        prob_i13 = parsed_vars.get("prob_i13")[opt_num][change_num]
        prob_f13 = parsed_vars.get("prob_f13")[opt_num][change_num]
        if num_pers == 0:
            # n_prop_destroy13 = How many effectively equivalent people would have their property damaged
            n_prop_change13 = parsed_vars.get("n_prop_change13")[opt_num][change_num]
            waiver13 = parsed_vars.get("waiver13")[opt_num][change_num]
            change = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
        else:
            n_prop_change13 = num_pers
            waiver13 = waiver
            change = prop_val / 4050000
        top_change_wt = (
            n_prop_change13
            * (prob_i13 - prob_f13)
            / 100
            * change
            * (1.0 - waiver13 / 100.0)
        )
    elif top_change == 14:
        # 14 - Repairing/beautifying property or decreasing/increasing its probability
        time_prob14 = parsed_vars.get("time_prob14")[opt_num][change_num]
        prob_i14 = parsed_vars.get("prob_i14")[opt_num][change_num]
        prob_f14 = parsed_vars.get("prob_f14")[opt_num][change_num]
        if num_pers == 0:
            # n_prop_repair14 = How many effectively equivalent people would have their property repaired
            n_prop_repair14 = parsed_vars.get("n_prop_repair14")[opt_num][change_num]
            waiver14 = parsed_vars.get("waiver14")[opt_num][change_num]
        else:
            n_prop_repair14 = num_pers
            waiver14 = waiver
        change = property_repair_fn(parsed_vars, opt_num, change_num)
        # For beautifying:
        # n_beauty14 = Number of people for which the beauty enhances their experience of life
        n_beauty14 = parsed_vars.get("n_beauty14")[opt_num][change_num]
        # qol_up14 = Average quality of life increase due to beauty (0 to 1)
        qol_up14 = parsed_vars.get("qol_up14")[opt_num][change_num]
        # days14 = Average time of the quality of life increase (in days)
        days14 = parsed_vars.get("days14")[opt_num][change_num]
        change_beaut = n_beauty14 * qol_up14 * days14 / 365 / 54
        top_change_wt = (
            n_prop_repair14
            * (prob_i14 - prob_f14)
            / 100
            * (change + change_beaut)
            * (1.0 - waiver14 / 100.0)
        )
    elif top_change == 15:
        # 15 - Returning something stolen or decreasing/increasing its probability
        time_prob15 = parsed_vars.get("time_prob15")[opt_num][change_num]
        prob_i15 = parsed_vars.get("prob_i15")[opt_num][change_num]
        prob_f15 = parsed_vars.get("prob_f15")[opt_num][change_num]
        top_change_wt = (
            return_stolen_fn(parsed_vars, opt_num, change_num)
            * (prob_i15 - prob_f15)
            / 100
        )
    elif top_change == 16:
        # 16 - Freely chosen anti-survival (masochistic) physical pain or increasing/decreasing its probability
        time_prob16 = parsed_vars.get("time_prob16")[opt_num][change_num]
        prob_i16 = parsed_vars.get("prob_i16")[opt_num][change_num]
        prob_f16 = parsed_vars.get("prob_f16")[opt_num][change_num]
        top_change_wt = (
            masochistic_pain_fn(parsed_vars, opt_num, change_num)
            * (prob_i16 - prob_f16)
            / 100
        )
    elif top_change == 17:
        # 17 - Anti-survival (sadistic) pleasure or increasing/decreasing its probability
        time_prob17 = parsed_vars.get("time_prob17")[opt_num][change_num]
        prob_i17 = parsed_vars.get("prob_i17")[opt_num][change_num]
        prob_f17 = parsed_vars.get("prob_f17")[opt_num][change_num]
        top_change_wt = (
            sadistic_pleasure_fn(parsed_vars, opt_num, change_num)
            * (prob_i17 - prob_f17)
            / 100
        )
    elif top_change == 18:
        # 18 - Going against one’s conscience (includes not helping when one could), or increasing/decreasing its probability
        time_prob18 = parsed_vars.get("time_prob18")[opt_num][change_num]
        prob_i18 = parsed_vars.get("prob_i18")[opt_num][change_num]
        prob_f18 = parsed_vars.get("prob_f18")[opt_num][change_num]
        if num_pers == 0:
            # n_consc18 = How many effectively equivalent people would go against their conscience
            n_consc18 = parsed_vars.get("n_consc18")[opt_num][change_num]
            waiver18 = parsed_vars.get("waiver18")[opt_num][change_num]
        else:
            n_consc18 = num_pers
            waiver18 = waiver
        change = conscience_fn(parsed_vars, opt_num, change_num)
        top_change_wt = (
            n_consc18 * (prob_i18 - prob_f18) / 100 * change * (1.0 - waiver18 / 100.0)
        )
    elif top_change == 19:
        # 19 - Denying responsibility, lowering one’s self-esteem, or increasing/decreasing its probability
        time_prob19 = parsed_vars.get("time_prob19")[opt_num][change_num]
        prob_i19 = parsed_vars.get("prob_i19")[opt_num][change_num]
        prob_f19 = parsed_vars.get("prob_f19")[opt_num][change_num]
        top_change_wt = (
            not_responsible_fn(parsed_vars, opt_num, change_num)
            * (prob_i19 - prob_f19)
            / 100
        )
    elif top_change == 20:
        # 20 - Taking responsibility, building one’s self-esteem, or decreasing/increasing its probability
        time_prob20 = parsed_vars.get("time_prob20")[opt_num][change_num]
        prob_i20 = parsed_vars.get("prob_i20")[opt_num][change_num]
        prob_f20 = parsed_vars.get("prob_f20")[opt_num][change_num]
        top_change_wt = (
            be_responsible_fn(parsed_vars, opt_num, change_num)
            * (prob_i20 - prob_f20)
            / 100
        )
    elif top_change == 21:
        # 21 - Thinking through the ethics of one’s decisions in advance or decreasing/increasing the probability of this
        time_prob21 = parsed_vars.get("time_prob21")[opt_num][change_num]
        prob_i21 = parsed_vars.get("prob_i21")[opt_num][change_num]
        prob_f21 = parsed_vars.get("prob_f21")[opt_num][change_num]
        top_change_wt = (
            prethought_fn(parsed_vars, opt_num, change_num)
            * (prob_i21 - prob_f21)
            / 100
        )
    elif top_change == 22:
        # 22 - Actively going against justice being upheld (denying due process), or increasing/decreasing its probability
        time_prob22 = parsed_vars.get("time_prob22")[opt_num][change_num]
        prob_i22 = parsed_vars.get("prob_i22")[opt_num][change_num]
        prob_f22 = parsed_vars.get("prob_f22")[opt_num][change_num]
        # For the person who did the injustice who isn't being inspired to take responsibility:
        not_responsible = not_responsible_fn(parsed_vars, opt_num, change_num)
        # For the person who's committing an ethical breach by not upholding justice:
        conscience = conscience_fn(parsed_vars, opt_num, change_num)
        top_change_wt = (not_responsible + conscience) * (prob_i22 - prob_f22) / 100
    elif top_change == 23:
        # 23 - Upholding justice (holding people responsible), or decreasing/increasing its probability
        time_prob23 = parsed_vars.get("time_prob23")[opt_num][change_num]
        prob_i23 = parsed_vars.get("prob_i23")[opt_num][change_num]
        prob_f23 = parsed_vars.get("prob_f23")[opt_num][change_num]
        responsible = be_responsible_fn(parsed_vars, opt_num, change_num)
        top_change_wt = responsible * (prob_i23 - prob_f23) / 100
    elif top_change == 24:
        # 24 - An animal dying or increasing/decreasing its probability
        time_prob24 = parsed_vars.get("time_prob24")[opt_num][change_num]
        prob_i24 = parsed_vars.get("prob_i24")[opt_num][change_num]
        prob_f24 = parsed_vars.get("prob_f24")[opt_num][change_num]
        top_change_wt = (
            animal_dying_fn(parsed_vars, opt_num, change_num)
            * (prob_i24 - prob_f24)
            / 100
        )
    elif top_change == 25:
        # 25 - Physical pain of animals or increasing/decreasing its probability
        time_prob25 = parsed_vars.get("time_prob25")[opt_num][change_num]
        prob_i25 = parsed_vars.get("prob_i25")[opt_num][change_num]
        prob_f25 = parsed_vars.get("prob_f25")[opt_num][change_num]
        top_change_wt = (
            animal_pain_fn(parsed_vars, opt_num, change_num)
            * (prob_i25 - prob_f25)
            / 100
        )
    elif top_change == 26:
        # 26 - Words or actions that encourage violence, or increasing/decreasing their probabilities
        time_prob26 = parsed_vars.get("time_prob26")[opt_num][change_num]
        prob_i26 = parsed_vars.get("prob_i26")[opt_num][change_num]
        prob_f26 = parsed_vars.get("prob_f26")[opt_num][change_num]
        # violence_up26 = How much more likely specific violence is made (in percent)
        violence_up26 = parsed_vars.get("violence_up26")[opt_num][change_num]
        # likely_life_loss26 = How likely specific violence is to lead to loss of life (in percent)
        likely_life_loss26 = parsed_vars.get("likely_life_loss26")[opt_num][change_num]
        # n_rtl_viol26 = How many rights to life violated
        n_rtl_viol26 = parsed_vars.get("n_rtl_viol26")[opt_num][change_num]
        life_val = life_value_fn(parsed_vars, opt_num, change_num)
        # likely_pain26 = How likely specific violence is to lead to pain (in percent)
        likely_pain26 = parsed_vars.get("likely_pain26")[opt_num][change_num]
        # n_rtbi_viol26 = How many rights to body integrity violated
        n_rtbi_viol26 = parsed_vars.get("n_rtbi_viol26")[opt_num][change_num]
        pain = unwanted_pain_fn(parsed_vars, opt_num, change_num)
        # likely_health26 = How likely specific violence is to lead to loss of function due to health issues (in percent)
        likely_health26 = parsed_vars.get("likely_health26")[opt_num][change_num]
        health = function_loss_fn(parsed_vars, opt_num, change_num)
        # likely_steal26 = How likely specific violence is to lead to stealing/damage to property (in percent)
        likely_steal26 = parsed_vars.get("likely_steal26")[opt_num][change_num]
        # n_rtp_viol26 = How many rights to property violated
        n_rtp_viol26 = parsed_vars.get("n_rtp_viol26")[opt_num][change_num]
        steal = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
        # perc_culp26 = The person's culpability for the words encouraging violence against them, in percent
        perc_culp26 = parsed_vars.get("perc_culp26")[opt_num][change_num]
        # violence_gen_up26 = How much more likely violence is made in general (in percent)
        violence_gen_up26 = parsed_vars.get("violence_gen_up26")[opt_num][change_num]
        # Estimated weight of damage from all violence in world (including violation of rights to life and body integrity)
        wt_all_violence = 10000000000.0  # 10 billion
        top_change_wt = (
            (
                violence_up26
                / 100
                * (
                    likely_life_loss26 / 100 * life_val
                    + likely_pain26 / 100 * pain
                    + likely_health26 / 100 * health
                    + likely_steal26 / 100 * steal
                )
                + (
                    likely_life_loss26 / 100 * n_rtl_viol26 * 10000
                    + likely_pain26
                    / 100
                    * n_rtbi_viol26
                    * (10000 * (pain + health) + 1)
                    + likely_steal26 / 100 * n_rtp_viol26 * (100 * steal + 1)
                )
                * (1 - perc_culp26 / 100)
                + wt_all_violence * violence_gen_up26 / 100
            )
            * (prob_i26 - prob_f26)
            / 100
        )
    elif top_change == 27:
        # 27 - Words or actions that inspire non-violence, discourage violence, or decreasing/increasing their probabilities
        time_prob27 = parsed_vars.get("time_prob27")[opt_num][change_num]
        prob_i27 = parsed_vars.get("prob_i27")[opt_num][change_num]
        prob_f27 = parsed_vars.get("prob_f27")[opt_num][change_num]
        # likely_violence27 = How much less likely specific violence is made (in percent)
        likely_violence27 = parsed_vars.get("likely_violence27")[opt_num][change_num]
        # likely_death27 = How likely specific violence would be to lead to loss of life (in percent)
        likely_death27 = parsed_vars.get("likely_death27")[opt_num][change_num]
        # n_rtl_viol27 = How many rights to life violated
        n_rtl_viol27 = parsed_vars.get("n_rtl_viol27")[opt_num][change_num]
        life_val = life_value_fn(parsed_vars, opt_num, change_num)
        # likely_pain27 = How likely specific violence would be to lead to pain (in percent)
        likely_pain27 = parsed_vars.get("likely_pain27")[opt_num][change_num]
        # n_rtbi_viol27 = How many rights to body integrity violated
        n_rtbi_viol27 = parsed_vars.get("n_rtbi_viol27")[opt_num][change_num]
        pain = unwanted_pain_fn(parsed_vars, opt_num, change_num)
        # likely_health27 = How likely specific violence would be to lead to loss of function due to health issues (in percent)
        likely_health27 = parsed_vars.get("likely_health27")[opt_num][change_num]
        health = function_loss_fn(parsed_vars, opt_num, change_num)
        # likely_steal27 = How likely specific violence would be to lead to stealing/damage to property (in percent)
        likely_steal27 = parsed_vars.get("likely_steal27")[opt_num][change_num]
        # n_rtp_viol27 = How many rights to property violated
        n_rtp_viol27 = parsed_vars.get("n_rtp_viol27")[opt_num][change_num]
        steal = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
        # likely_violence_gen27 = How much less likely violence is made in general (in percent)
        likely_violence_gen27 = parsed_vars.get("likely_violence_gen27")[opt_num][
            change_num
        ]
        # Estimated weight of damage from all violence in world (including violation of rights to life and body integrity)
        violence_gen = 10000000000.0  # 10 billion
        top_change_wt = (
            (
                likely_violence27
                / 100
                * (
                    likely_death27 / 100 * life_val
                    + likely_pain27 / 100 * pain
                    + likely_health27 / 100 * health
                    + likely_steal27 / 100 * steal
                )
                + (
                    likely_death27 / 100 * n_rtl_viol27 * 10000
                    + n_rtbi_viol27
                    * (
                        10000
                        * (likely_pain27 / 100 * pain + likely_health27 / 100 * health)
                        + 1
                    )
                    + likely_steal27 / 100 * n_rtp_viol27 * (100 * steal + 1)
                )
                + violence_gen * likely_violence_gen27 / 100
            )
            * (prob_i27 - prob_f27)
            / 100
        )
    elif top_change == 28:
        # 28 - Words or actions that encourage stealing, or increasing/decreasing their probabilities
        time_prob28 = parsed_vars.get("time_prob28")[opt_num][change_num]
        prob_i28 = parsed_vars.get("prob_i28")[opt_num][change_num]
        prob_f28 = parsed_vars.get("prob_f28")[opt_num][change_num]
        # steal_up28 = How much more likely specific stealing is made (in percent)
        steal_up28 = parsed_vars.get("steal_up28")[opt_num][change_num]
        steal = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
        # perc_culp28 = The person's culpability for the words/actions encouraging stealing, in percent
        perc_culp28 = parsed_vars.get("perc_culp28")[opt_num][change_num]
        # steal_gen_up28 = How much more likely stealing is made in general (in percent)
        steal_gen_up28 = parsed_vars.get("steal_gen_up28")[opt_num][change_num]
        # Estimated weight of damage from all stealing in world (including violation of property rights)
        wt_all_steal = 1000000.0
        # This doesn't take into account that stealing could be from more than one person
        top_change_wt = (
            (
                steal * steal_up28 / 100
                + wt_all_steal * steal_gen_up28 / 100
                + steal_up28 * (100.0 * steal + 1.0) * (1.0 - perc_culp28 / 100.0)
            )
            * (prob_i28 - prob_f28)
            / 100
        )
    elif top_change == 29:
        # 29 - Words or actions that inspire earning what you get, discourage stealing, or decreasing/increasing their probabilities
        time_prob29 = parsed_vars.get("time_prob29")[opt_num][change_num]
        prob_i29 = parsed_vars.get("prob_i29")[opt_num][change_num]
        prob_f29 = parsed_vars.get("prob_f29")[opt_num][change_num]
        # likely_steal29 = How much less likely specific stealing is made (in percent)
        likely_steal29 = parsed_vars.get("likely_steal29")[opt_num][change_num]
        steal = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
        # likely_steal_gen29 = How much less likely stealing is made in general (in percent)
        likely_steal_gen29 = parsed_vars.get("likely_steal_gen29")[opt_num][change_num]
        # Estimated weight of damage from all stealing in world (including violation of property rights)
        steal_gen = 1000000.0
        top_change_wt = (
            (
                steal * likely_steal29 / 100
                + steal_gen * likely_steal_gen29 / 100
                + likely_steal29 / 100 * (100.0 * steal + 1.0)
            )
            * (prob_i29 - prob_f29)
            / 100
        )
    elif top_change == 30:
        # 30 - Words that spread false info (including misrepresenting the hierarchy of value), or increasing/decreasing their probabilities
        time_prob30 = parsed_vars.get("time_prob30")[opt_num][change_num]
        prob_i30 = parsed_vars.get("prob_i30")[opt_num][change_num]
        prob_f30 = parsed_vars.get("prob_f30")[opt_num][change_num]
        # destroy_up30 = How much more likely specific value destruction is made (in percent)
        destroy_up30 = parsed_vars.get("destroy_up30")[opt_num][change_num]
        # wt_destroy30 = Weight of specific value destruction
        wt_destroy30 = parsed_vars.get("wt_destroy30")[opt_num][change_num]
        # trust_down_gen30 = How much the words erode trust in general (in percent)
        trust_down_gen30 = parsed_vars.get("trust_down_gen30")[opt_num][change_num]
        # wt_destroy_gen30 = Weight of value destruction due to general erosion of trust
        wt_destroy_gen30 = parsed_vars.get("wt_destroy_gen30")[opt_num][change_num]
        top_change_wt = (
            (
                wt_destroy30 * destroy_up30 / 100
                + wt_destroy_gen30 * trust_down_gen30 / 100
            )
            * (prob_i30 - prob_f30)
            / 100
        )
    elif top_change == 31:
        # 31 - Words that correct false info (including accurately representing the hierarchy of value), or decreasing/increasing their probabilities
        time_prob31 = parsed_vars.get("time_prob31")[opt_num][change_num]
        prob_i31 = parsed_vars.get("prob_i31")[opt_num][change_num]
        prob_f31 = parsed_vars.get("prob_f31")[opt_num][change_num]
        # likely_destroy31 = How much less likely specific value destruction is made (in percent)
        likely_destroy31 = parsed_vars.get("likely_destroy31")[opt_num][change_num]
        # wt_destroy31 = Weight of specific value destruction
        wt_destroy31 = parsed_vars.get("wt_destroy31")[opt_num][change_num]
        # likely_build31 = How much the words help mend trust in general (in percent)
        likely_build31 = parsed_vars.get("likely_build31")[opt_num][change_num]
        # wt_build31 = Weight of value build due to general gain of trust
        wt_build31 = parsed_vars.get("wt_build31")[opt_num][change_num]
        top_change_wt = (
            (wt_destroy31 * likely_destroy31 / 100 + wt_build31 * likely_build31 / 100)
            * (prob_i31 - prob_f31)
            / 100
        )
    elif top_change == 32:
        # 32 - Actions that misrepresent the hierarchy of value, or increasing/decreasing their probabilities
        time_prob32 = parsed_vars.get("time_prob32")[opt_num][change_num]
        prob_i32 = parsed_vars.get("prob_i32")[opt_num][change_num]
        prob_f32 = parsed_vars.get("prob_f32")[opt_num][change_num]
        # destroy_up32 = How much more likely specific value destruction is made (in percent)
        destroy_up32 = parsed_vars.get("destroy_up32")[opt_num][change_num]
        # wt_destroy32 = Weight of specific value destruction
        wt_destroy32 = parsed_vars.get("wt_destroy32")[opt_num][change_num]
        # wt_destroy_gen32 = How much the actions erode trust in general (in percent)
        wt_destroy_gen32 = parsed_vars.get("wt_destroy_gen32")[opt_num][change_num]
        # wt_destroy_trust32 = Weight of value destruction due to general erosion of trust
        wt_destroy_trust32 = parsed_vars.get("wt_destroy_trust32")[opt_num][change_num]
        top_change_wt = (
            (
                wt_destroy32 * destroy_up32 / 100
                + wt_destroy_trust32 * wt_destroy_gen32 / 100
            )
            * (prob_i32 - prob_f32)
            / 100
        )
    elif top_change == 33:
        # 33 - Actions that accurately represent the hierarchy of value, or decreasing/increasing their probabilities
        time_prob33 = parsed_vars.get("time_prob33")[opt_num][change_num]
        prob_i33 = parsed_vars.get("prob_i33")[opt_num][change_num]
        prob_f33 = parsed_vars.get("prob_f33")[opt_num][change_num]
        # likely_destroy33 = How much less likely specific value destruction is made (in percent)
        likely_destroy33 = parsed_vars.get("likely_destroy33")[opt_num][change_num]
        # wt_destroy33 = Weight of specific value destruction
        wt_destroy33 = parsed_vars.get("wt_destroy33")[opt_num][change_num]
        # likely_build33 = How much the actions support trust in general (in percent)
        likely_build33 = parsed_vars.get("likely_build33")[opt_num][change_num]
        # wt_build33 = Weight of value build due to general gain of trust
        wt_build33 = parsed_vars.get("wt_build33")[opt_num][change_num]
        top_change_wt = (
            (wt_destroy33 * likely_destroy33 / 100 + wt_build33 * likely_build33 / 100)
            * (prob_i33 - prob_f33)
            / 100
        )
    elif top_change == 34:
        # 34 - Words or actions that discourage empathy, creativity, curiosity, critical thinking, honest effort and/or responsibility, or increasing/decreasing their probabilities
        time_prob34 = parsed_vars.get("time_prob34")[opt_num][change_num]
        prob_i34 = parsed_vars.get("prob_i34")[opt_num][change_num]
        prob_f34 = parsed_vars.get("prob_f34")[opt_num][change_num]
        # build_down34 = How much less likely value is to be built (in percent)
        build_down34 = parsed_vars.get("build_down34")[opt_num][change_num]
        # wt_would_build34 = Weight of value that would've otherwise been built
        wt_would_build34 = parsed_vars.get("wt_would_build34")[opt_num][change_num]
        # destroy_up34 = How much more likely value is to be destroyed (in percent)
        destroy_up34 = parsed_vars.get("destroy_up34")[opt_num][change_num]
        # wt_would_destroy34 = Weight of value that would be destroyed
        wt_would_destroy34 = parsed_vars.get("wt_would_destroy34")[opt_num][change_num]
        top_change_wt = (
            (
                wt_would_destroy34 * destroy_up34 / 100
                + wt_would_build34 * build_down34 / 100
            )
            * (prob_i34 - prob_f34)
            / 100
        )
    elif top_change == 35:
        # 35 - Words or actions that encourage empathy, creativity, curiosity, critical thinking, honest effort, and/or responsibility, or decreasing/increasing their probabilities
        time_prob35 = parsed_vars.get("time_prob35")[opt_num][change_num]
        prob_i35 = parsed_vars.get("prob_i35")[opt_num][change_num]
        prob_f35 = parsed_vars.get("prob_f35")[opt_num][change_num]
        # likely_destroy35 = How much less likely value is to be destroyed (in percent)
        likely_destroy35 = parsed_vars.get("likely_destroy35")[opt_num][change_num]
        # wt_destroy35 = Weight of value that could've been destroyed
        wt_destroy35 = parsed_vars.get("wt_destroy35")[opt_num][change_num]
        # likely_build35 = How much more likely value is to be built (in percent)
        likely_build35 = parsed_vars.get("likely_build35")[opt_num][change_num]
        # wt_build35 = Weight of increased value that could be built
        wt_build35 = parsed_vars.get("wt_build35")[opt_num][change_num]
        top_change_wt = (
            (wt_destroy35 * likely_destroy35 / 100 + wt_build35 * likely_build35 / 100)
            * (prob_i35 - prob_f35)
            / 100
        )
    elif top_change == 36:
        # 36 - A plant dying, or increasing/decreasing its probability
        time_prob36 = parsed_vars.get("time_prob36")[opt_num][change_num]
        prob_i36 = parsed_vars.get("prob_i36")[opt_num][change_num]
        prob_f36 = parsed_vars.get("prob_f36")[opt_num][change_num]
        top_change_wt = (
            plant_dying_fn(parsed_vars, opt_num, change_num)
            * (prob_i36 - prob_f36)
            / 100
        )
    elif top_change == 37:
        # 37 - Errors of thought, or increasing/decreasing their probabilities
        # Levels of thinking: 1 - completely mindless, 2 - quickly jumping to conclusions,
        # 3 - emotion-influenced “reasoning,” 4 - somewhat careful consideration of some options,
        # 5 - careful consideration via the scientific method, exploring many possibilities, putting emotions aside
        time_prob37 = parsed_vars.get("time_prob37")[opt_num][change_num]
        prob_i37 = parsed_vars.get("prob_i37")[opt_num][change_num]
        prob_f37 = parsed_vars.get("prob_f37")[opt_num][change_num]
        # think_level37 = Level of thinking used (1 to 5)
        think_level37 = parsed_vars.get("think_level37")[opt_num][change_num]
        # wt_destroy37 = How much value destruction weight is on the line
        wt_change37 = parsed_vars.get("wt_change37")[opt_num][change_num]
        # Assume increases in likelihood of destruction by 50% for thinking level 1, 20% for 2, 10% for 3, 1% for 4, and 0% for 5
        if think_level37 == 1:
            top_change_wt = wt_change37 * 0.5 * (prob_i37 - prob_f37) / 100
        elif think_level37 == 2:
            top_change_wt = wt_change37 * 0.2 * (prob_i37 - prob_f37) / 100
        elif think_level37 == 3:
            top_change_wt = wt_change37 * 0.1 * (prob_i37 - prob_f37) / 100
        elif think_level37 == 4:
            top_change_wt = wt_change37 * 0.01 * (prob_i37 - prob_f37) / 100
        elif think_level37 == 5:
            top_change_wt = 0 * (prob_i37 - prob_f37) / 100
    elif top_change == 38:
        # 38 - Practicing critical thinking, learning, or developing skills to increase one’s options, or decreasing/increasing its probability
        time_prob38 = parsed_vars.get("time_prob38")[opt_num][change_num]
        prob_i38 = parsed_vars.get("prob_i38")[opt_num][change_num]
        prob_f38 = parsed_vars.get("prob_f38")[opt_num][change_num]
        # difficulty38 = Level of difficulty (1 to 5)
        difficulty38 = parsed_vars.get("difficulty38")[opt_num][change_num]
        # minutes38 = Practice time (minutes)
        minutes38 = parsed_vars.get("minutes38")[opt_num][change_num]
        if difficulty38 == 1:
            top_change_wt = 0.00000001 * minutes38 * (prob_i38 - prob_f38) / 100
        elif difficulty38 == 2:
            top_change_wt = 0.0000001 * minutes38 * (prob_i38 - prob_f38) / 100
        elif difficulty38 == 3:
            top_change_wt = 0.000001 * minutes38 * (prob_i38 - prob_f38) / 100
        elif difficulty38 == 4:
            top_change_wt = 0.00001 * minutes38 * (prob_i38 - prob_f38) / 100
        elif difficulty38 == 5:
            top_change_wt = 0.0001 * minutes38 * (prob_i38 - prob_f38) / 100
    elif top_change == 39:
        # 39 - Discouraging human interaction, community, or increasing/decreasing its probability
        time_prob39 = parsed_vars.get("time_prob39")[opt_num][change_num]
        prob_i39 = parsed_vars.get("prob_i39")[opt_num][change_num]
        prob_f39 = parsed_vars.get("prob_f39")[opt_num][change_num]
        # qol_down39 = Average quality of life decrease per human (0 to 1)
        qol_down39 = parsed_vars.get("qol_down39")[opt_num][change_num]
        # minutes39 = Average time of quality of life decrease (minutes)
        minutes39 = parsed_vars.get("minutes39")[opt_num][change_num]
        # n_hum39 = How many humans affected
        n_hum39 = parsed_vars.get("n_hum39")[opt_num][change_num]
        top_change_wt = (
            qol_down39
            * minutes39
            / 60
            / 24
            / 365
            / 54
            * n_hum39
            * (prob_i39 - prob_f39)
            / 100
        )
    elif top_change == 40:
        # 40 - Promoting human interaction, community, or decreasing/increasing its probability
        time_prob40 = parsed_vars.get("time_prob40")[opt_num][change_num]
        prob_i40 = parsed_vars.get("prob_i40")[opt_num][change_num]
        prob_f40 = parsed_vars.get("prob_f40")[opt_num][change_num]
        # qol_up40 = Average quality of life increase per human (0 to 1)
        qol_up40 = parsed_vars.get("qol_up40")[opt_num][change_num]
        # minutes40 = Average time of quality of life increase (minutes)
        minutes40 = parsed_vars.get("minutes40")[opt_num][change_num]
        # n_hum40 = How many humans affected
        n_hum40 = parsed_vars.get("n_hum40")[opt_num][change_num]
        top_change_wt = (
            qol_up40
            * minutes40
            / 60
            / 24
            / 365
            / 54
            * n_hum40
            * (prob_i40 - prob_f40)
            / 100
        )
    elif top_change == 41:
        # 41 - Decreasing economic activity, or increasing/decreasing its probability
        time_prob41 = parsed_vars.get("time_prob41")[opt_num][change_num]
        prob_i41 = parsed_vars.get("prob_i41")[opt_num][change_num]
        prob_f41 = parsed_vars.get("prob_f41")[opt_num][change_num]
        # econ_down41 = Decreased economic activity ($) amount
        econ_down41 = parsed_vars.get("econ_down41")[opt_num][change_num]
        top_change_wt = econ_down41 / 4050000 * (prob_i41 - prob_f41) / 100
    elif top_change == 42:
        # 42 - Increasing economic activity, paying people to do work, or decreasing/increasing its probability
        time_prob42 = parsed_vars.get("time_prob42")[opt_num][change_num]
        prob_i42 = parsed_vars.get("prob_i42")[opt_num][change_num]
        prob_f42 = parsed_vars.get("prob_f42")[opt_num][change_num]
        # econ_up42 = Increased economic activity ($) amount
        econ_up42 = parsed_vars.get("econ_up42")[opt_num][change_num]
        top_change_wt = econ_up42 / 4050000 * (prob_i42 - prob_f42) / 100
    elif top_change == 43:
        # 43 - Reducing options to net build value, or increasing/decreasing its probability
        time_prob43 = parsed_vars.get("time_prob43")[opt_num][change_num]
        prob_i43 = parsed_vars.get("prob_i43")[opt_num][change_num]
        prob_f43 = parsed_vars.get("prob_f43")[opt_num][change_num]
        # n_optdown43 = Number of people that options have been reduced for
        n_optdown43 = parsed_vars.get("n_optdown43")[opt_num][change_num]
        # enjoy_level_there43 = Average enjoyment level with options still there
        enjoy_level_there43 = parsed_vars.get("enjoy_level_there43")[opt_num][
            change_num
        ]
        # enjoy_level_gone43 = Average enjoyment level with options gone
        enjoy_level_gone43 = parsed_vars.get("enjoy_level_gone43")[opt_num][change_num]
        # minutes43 = Duration of enjoyment level drop (in minutes)
        minutes43 = parsed_vars.get("minutes43")[opt_num][change_num]
        # econ_decrease43 = Reduced economic activity ($) due to reduced options
        econ_decrease43 = parsed_vars.get("econ_decrease43")[opt_num][change_num]
        top_change_wt = (
            (
                n_optdown43
                * (1.903 * 10**-17)
                * minutes43
                * (10**enjoy_level_there43 - 10**enjoy_level_gone43)
                + econ_decrease43 / 4050000
            )
            * (prob_i43 - prob_f43)
            / 100
        )
    elif top_change == 44:
        # 44 - Increasing options to net build value, or decreasing/increasing its probability
        time_prob44 = parsed_vars.get("time_prob44")[opt_num][change_num]
        prob_i44 = parsed_vars.get("prob_i44")[opt_num][change_num]
        prob_f44 = parsed_vars.get("prob_f44")[opt_num][change_num]
        # n_hum44 = Number of people that options have been increased for
        n_hum44 = parsed_vars.get("n_hum44")[opt_num][change_num]
        # enjoy_level_opts44 = Average enjoyment level with increased options
        enjoy_level_opts44 = parsed_vars.get("enjoy_level_opts44")[opt_num][change_num]
        # enjoy_level_no_opts44 = Average enjoyment level without increased options
        enjoy_level_no_opts44 = parsed_vars.get("enjoy_level_no_opts44")[opt_num][
            change_num
        ]
        # minutes44 = Duration of enjoyment level increase (in minutes)
        minutes44 = parsed_vars.get("minutes44")[opt_num][change_num]
        # econ_up44 = Increased economic activity ($) due to increased options
        econ_up44 = parsed_vars.get("econ_up44")[opt_num][change_num]
        top_change_wt = (
            (
                n_hum44
                * (1.903 * 10**-17)
                * minutes44
                * (10**enjoy_level_opts44 - 10**enjoy_level_no_opts44)
                + econ_up44 / 4050000
            )
            * (prob_i44 - prob_f44)
            / 100
        )
    elif top_change == 45:
        # 45 - Putting in effort towards a net destructive goal, or increasing/decreasing its probability
        time_prob45 = parsed_vars.get("time_prob45")[opt_num][change_num]
        prob_i45 = parsed_vars.get("prob_i45")[opt_num][change_num]
        prob_f45 = parsed_vars.get("prob_f45")[opt_num][change_num]
        # wt_destroy45 = How destructive the goal is (value destruction weight)
        wt_destroy45 = parsed_vars.get("wt_destroy45")[opt_num][change_num]
        # goal_increase45 = How much effort increases likelihood of achieving goal (in percent)
        goal_increase45 = parsed_vars.get("goal_increase45")[opt_num][change_num]
        top_change_wt = (
            wt_destroy45 * goal_increase45 / 100 * (prob_i45 - prob_f45) / 100
        )
    elif top_change == 46:
        # 46 - Putting in effort towards a net non-destructive goal, or decreasing/increasing its probability
        time_prob46 = parsed_vars.get("time_prob46")[opt_num][change_num]
        prob_i46 = parsed_vars.get("prob_i46")[opt_num][change_num]
        prob_f46 = parsed_vars.get("prob_f46")[opt_num][change_num]
        # wt_build46 = How net value-building the goal is (value build weight)
        wt_build46 = parsed_vars.get("wt_build46")[opt_num][change_num]
        # likely_goal46 = How much effort increases likelihood of achieving goal (in percent)
        likely_goal46 = parsed_vars.get("likely_goal46")[opt_num][change_num]
        top_change_wt = wt_build46 * likely_goal46 / 100 * (prob_i46 - prob_f46) / 100
    elif top_change == 47:
        # 47 - Setting a bad example, or increasing/decreasing its probability
        time_prob47 = parsed_vars.get("time_prob47")[opt_num][change_num]
        prob_i47 = parsed_vars.get("prob_i47")[opt_num][change_num]
        prob_f47 = parsed_vars.get("prob_f47")[opt_num][change_num]
        # build_decrease47 = How much less likely value is to be built (in percent)
        build_decrease47 = parsed_vars.get("build_decrease47")[opt_num][change_num]
        # wt_build47 = Weight of value that would've otherwise been built
        wt_build47 = parsed_vars.get("wt_build47")[opt_num][change_num]
        # destroy_increase47 = How much more likely value is to be destroyed (in percent)
        destroy_increase47 = parsed_vars.get("destroy_increase47")[opt_num][change_num]
        # wt_destroy47 = Weight of value that would be destroyed
        wt_destroy47 = parsed_vars.get("wt_destroy47")[opt_num][change_num]
        top_change_wt = (
            (
                wt_destroy47 * destroy_increase47 / 100
                + wt_build47 * (1 - build_decrease47 / 100)
            )
            * (prob_i47 - prob_f47)
            / 100
        )
    elif top_change == 48:
        # 48 - Setting a good example and inspiring others, or decreasing/increasing its probability
        time_prob48 = parsed_vars.get("time_prob48")[opt_num][change_num]
        prob_i48 = parsed_vars.get("prob_i48")[opt_num][change_num]
        prob_f48 = parsed_vars.get("prob_f48")[opt_num][change_num]
        # likely_build48 = How much more likely value is to be built (in percent)
        likely_build48 = parsed_vars.get("likely_build48")[opt_num][change_num]
        # wt_build48 = Weight of value that would be built
        wt_build48 = parsed_vars.get("wt_build48")[opt_num][change_num]
        # likely_destroy48 = How much less likely value is to be destroyed (in percent)
        likely_destroy48 = parsed_vars.get("likely_destroy48")[opt_num][change_num]
        # wt_destroy48 = Weight of value that would've been destroyed
        wt_destroy48 = parsed_vars.get("wt_destroy48")[opt_num][change_num]
        top_change_wt = (
            (
                wt_build48 * likely_build48 / 100
                + wt_destroy48 * (1 - likely_destroy48 / 100)
            )
            * (prob_i48 - prob_f48)
            / 100
        )
    elif top_change == 49:
        # 49 - Being creative in art or science, or decreasing/increasing its probability
        time_prob49 = parsed_vars.get("time_prob49")[opt_num][change_num]
        prob_i49 = parsed_vars.get("prob_i49")[opt_num][change_num]
        prob_f49 = parsed_vars.get("prob_f49")[opt_num][change_num]
        # art_or_sci49 = For art (A) or science (S)?
        art_or_sci49 = parsed_vars.get("art_or_sci49")[opt_num][change_num]
        # skill_level49 = Creative at what skill level (1 to 5)?
        skill_level49 = parsed_vars.get("skill_level49")[opt_num][change_num]
        if art_or_sci49 == "A":
            # wt_art49 = Weight of beauty of art
            wt_art49 = parsed_vars.get("wt_art49")[opt_num][change_num]
            # n_art49 = How many people affected by beauty of art
            n_art49 = parsed_vars.get("n_art49")[opt_num][change_num]
            top_change_wt = n_art49 * wt_art49 * (prob_i49 - prob_f49) / 100
        else:
            # wt_science49 = Weight of scientific discovery
            wt_science49 = parsed_vars.get("wt_science49")[opt_num][change_num]
            top_change_wt = wt_science49 * (prob_i49 - prob_f49) / 100
    elif top_change == 50:
        # 50 - Giving yourself or someone else pleasure/new experiences that are welcomed, or decreasing/increasing its probability
        time_prob50 = parsed_vars.get("time_prob50")[opt_num][change_num]
        prob_i50 = parsed_vars.get("prob_i50")[opt_num][change_num]
        prob_f50 = parsed_vars.get("prob_f50")[opt_num][change_num]
        top_change_wt = (
            pleasure_fn(parsed_vars, opt_num, change_num) * (prob_i50 - prob_f50) / 100
        )
    elif top_change == 51:
        # 51 - Cooperating with others, or decreasing/increasing its probability
        time_prob51 = parsed_vars.get("time_prob51")[opt_num][change_num]
        prob_i51 = parsed_vars.get("prob_i51")[opt_num][change_num]
        prob_f51 = parsed_vars.get("prob_f51")[opt_num][change_num]
        # build_more51 = How much value build weight could be built through cooperation
        build_more51 = parsed_vars.get("build_more51")[opt_num][change_num]
        # likely_build51 = How much more likely that value is to be built with cooperation (in percent)
        likely_build51 = parsed_vars.get("likely_build51")[opt_num][change_num]
        top_change_wt = (
            build_more51 * likely_build51 / 100 * (prob_i51 - prob_f51) / 100
        )
    elif top_change == 52:
        # 52 - Helping others, or decreasing/increasing its probability
        time_prob52 = parsed_vars.get("time_prob52")[opt_num][change_num]
        prob_i52 = parsed_vars.get("prob_i52")[opt_num][change_num]
        prob_f52 = parsed_vars.get("prob_f52")[opt_num][change_num]
        # n_helped52 = How many people affected
        n_helped52 = parsed_vars.get("n_helped52")[opt_num][change_num]
        # qol_increase52 = Average quality of life increase
        qol_increase52 = parsed_vars.get("qol_increase52")[opt_num][change_num]
        # hours52 = Average time quality of life increase lasts (in hours)
        hours52 = parsed_vars.get("hours52")[opt_num][change_num]
        top_change_wt = (
            n_helped52
            * qol_increase52
            * hours52
            / 24
            / 365
            / 54
            * (prob_i52 - prob_f52)
            / 100
        )
    elif top_change == 53:
        # 53 - Violating right to life, or increasing/decreasing its probability
        time_prob53 = parsed_vars.get("time_prob53")[opt_num][change_num]
        prob_i53 = parsed_vars.get("prob_i53")[opt_num][change_num]
        prob_f53 = parsed_vars.get("prob_f53")[opt_num][change_num]
        # villain = Did this person put anyone in harm’s way (0 for no, 1 for yes)?
        villain = parsed_vars.get("villainy")[person_num - 1]
        # villain_prop = Did this person put anyone’s property in harm’s way (0 for no, 1 for yes)?
        villain_prop = parsed_vars.get("villainy_prop")[person_num - 1]
        # percent_culp = What's their percent culpability for getting themselves in harm's way?
        percent_culp = parsed_vars.get("percent_culpability")[person_num - 1]
        # harms = What percent of harm's way are they in?
        harms = parsed_vars.get("harms_way")[person_num - 1]
        # num_pers = How many people are equivalent to this person, including them themselves?
        num_pers = parsed_vars.get("num_persons")[person_num - 1]
        top_change_wt = (
            life_rights_fn(
                parsed_vars,
                villain,
                villain_prop,
                percent_culp,
                harms,
                num_pers,
                opt_num,
                change_num,
            )
            * (prob_i53 - prob_f53)
            / 100
        )
        rights_change_wt = top_change_wt
    elif top_change == 54:
        # 54 - Violating right to body integrity, or increasing/decreasing its probability
        time_prob54 = parsed_vars.get("time_prob54")[opt_num][change_num]
        prob_i54 = parsed_vars.get("prob_i54")[opt_num][change_num]
        prob_f54 = parsed_vars.get("prob_f54")[opt_num][change_num]
        # villain = Did this person put anyone in harm’s way (0 for no, 1 for yes)?
        villain = parsed_vars.get("villainy")[person_num - 1]
        # villain_prop = Did this person put anyone’s property in harm’s way (0 for no, 1 for yes)?
        villain_prop = parsed_vars.get("villainy_prop")[person_num - 1]
        # percent_culp = What's their percent culpability for getting themselves in harm's way?
        percent_culp = parsed_vars.get("percent_culpability")[person_num - 1]
        # harms = What percent of harm's way are they in?
        harms = parsed_vars.get("harms_way")[person_num - 1]
        # num_pers = How many people are equivalent to this person, including them themselves?
        num_pers = parsed_vars.get("num_persons")[person_num - 1]
        (
            top_change_wt,
            pain_o,
            health_o,
            threat_pain_o,
            threat_health_o,
            threat_kill_o,
            restraint_o,
            rights_change_wt,
        ) = body_rights_fn(
            parsed_vars,
            villain,
            villain_prop,
            percent_culp,
            harms,
            num_pers,
            opt_num,
            change_num,
        )
        top_change_wt = top_change_wt * (prob_i54 - prob_f54) / 100
        rights_change_wt = rights_change_wt * (prob_i54 - prob_f54) / 100
    elif top_change == 55:
        # 55 - Violating right to property, or increasing/decreasing its probability
        time_prob55 = parsed_vars.get("time_prob55")[opt_num][change_num]
        prob_i55 = parsed_vars.get("prob_i55")[opt_num][change_num]
        prob_f55 = parsed_vars.get("prob_f55")[opt_num][change_num]
        # villain = Did this person put anyone in harm’s way (0 for no, 1 for yes)?
        villain = parsed_vars.get("villainy")[person_num - 1]
        # villain_prop = Did this person put anyone’s property in harm’s way (0 for no, 1 for yes)?
        villain_prop = parsed_vars.get("villainy_prop")[person_num - 1]
        # culp_prop = What's their percent culpability for getting their property in harm's way?
        culp_prop = parsed_vars.get("culp_prop", [])[person_num - 1]
        # harms_prop = What percent of harm's way is their property in?
        harms_prop = parsed_vars.get("harms_way_prop")[person_num - 1]
        # num_pers = How many people are equivalent to this person, including them themselves?
        num_pers = parsed_vars.get("num_persons")[person_num - 1]
        # prop_val = Value weight of property calculated from steal_and_prop_destroy_fn
        prop_val = parsed_vars.get("prop_value")[person_num - 1]
        prop_rights_o, prop_value_o, top_change_wt, rights_change_wt = prop_rights_fn(
            parsed_vars,
            villain,
            villain_prop,
            culp_prop,
            harms_prop,
            prop_val,
            num_pers,
            opt_num,
            change_num,
        )
        top_change_wt = top_change_wt * (prob_i55 - prob_f55) / 100
        rights_change_wt = rights_change_wt * (prob_i55 - prob_f55) / 100
    print(value_change_output, top_change_wt)
    return top_change_wt, rights_change_wt

In [3]:
def life_value_fn(parsed_vars, opt_num, change_num):
    # This is a very rough and incomplete approximation to life value for these proof-of-concept calculations
    # age1f1 = Age (in years) of person
    age1f1 = parsed_vars.get("age1f1")[opt_num][change_num]
    # sex1f1 = Sex of person (M for male or F for female)
    sex1f1 = parsed_vars.get("sex1f1")[opt_num][change_num]
    # qol1f1 = Average quality of remaining life (-1 to 1)
    qol1f1 = parsed_vars.get("qol1f1")[opt_num][change_num]
    # income1f1 = Yearly income (in $) of person
    income1f1 = parsed_vars.get("income1f1")[opt_num][change_num]
    # unpaid_income1f1 = parsed_vars.get('unpaid_income1f1')[opt_num][change_num]
    # Equivalent yearly income ($) if were paid for unpaid labor
    # n_interact1f1 = Average number of people per year they would interact with if they live
    n_interact1f1 = parsed_vars.get("n_interact1f1")[opt_num][change_num]
    # value_add1f1 = Average value add per person they would interact with
    value_add1f1 = parsed_vars.get("value_add1f1")[opt_num][change_num]
    # n_loss1f1 = How many relationships they have with people who would feel the emotional loss
    n_loss1f1 = parsed_vars.get("n_loss1f1")[opt_num][change_num]
    # emot_depth1f1 = Average depth of relationships, for the emotional loss felt if they die (0 to 1)
    emot_depth1f1 = parsed_vars.get("emot_depth1f1")[opt_num][change_num]
    if sex1f1 == "F":
        if age1f1 <= 30:
            reproduce = 2
        elif age1f1 < 50:
            reproduce = (
                0.000333 * age1f1**3
                - 0.03926 * age1f1**2
                + 1.4095 * age1f1
                - 13.943
            )
        else:
            reproduce = 0
        # From curve fit to data from https://www.ssa.gov/oact/STATS/table4c6.html, female:
        life_expect = (
            4.423 * 10**-5 * age1f1**3
            - 0.0031 * age1f1**2
            + 0.092 * age1f1
            + 79.72
        )
    else:
        if age1f1 <= 47:
            reproduce = 0.005
        elif age1f1 <= 90:
            reproduce = 9.828 * 10**24 * age1f1**-16.34
        else:
            reproduce = 0
        # From curve fit to data from https://www.ssa.gov/oact/STATS/table4c6.html, male:
        life_expect = (
            3.49 * 10**-5 * age1f1**3
            - 0.0014 * age1f1**2
            + 0.0613 * age1f1
            + 74.24
        )
    # print(f"Life expectancy calculated to be {life_expect} years.")
    if age1f1 < 66:
        life_val = (
            (65 - age1f1) * income1f1 / 4050000
            + (life_expect - age1f1) * (qol1f1 / 54 + n_interact1f1 * value_add1f1)
            + 0.001 * emot_depth1f1 * n_loss1f1
            + reproduce
        )
    elif age1f1 < 71:
        life_val = (
            (life_expect - age1f1) * (qol1f1 / 54 + n_interact1f1 * value_add1f1)
            + 0.001 * emot_depth1f1 * n_loss1f1
            + reproduce
        )
    else:
        life_val = (
            (life_expect - age1f1) * (qol1f1 / 54 + n_interact1f1 * value_add1f1)
            + 0.0001 * emot_depth1f1 * n_loss1f1
            + reproduce
        )
    # print(f"*****Life value is: {life_val}")
    return life_val


def unwanted_pain_fn(parsed_vars, opt_num, change_num):
    # print(pain_levels)
    # pain_level1f2 = Physical pain level (0 to 10)
    pain_level1f2 = parsed_vars.get("pain_level1f2")[opt_num][change_num]
    if pain_level1f2 != 0:
        # duration1f2 = Duration of physical pain (in minutes)
        duration1f2 = parsed_vars.get("duration1f2")[opt_num][change_num]
        # unbuilt_value1f2 = Net value could've been built (in $) if not for unwanted pain
        unbuilt_value1f2 = parsed_vars.get("unbuilt_value1f2")[opt_num][change_num]
        if pain_level1f2 > 7:
            change_wt = (1.903 * 10**-6) * (
                duration1f2 * 100 ** (pain_level1f2 - 7)
            ) + unbuilt_value1f2 / 4050000.0
        else:
            change_wt = (1.903 * 10**-13) * (
                duration1f2 * 10**pain_level1f2
            ) + unbuilt_value1f2 / 4050000.0
    else:
        change_wt = 0.0
    return change_wt


def function_loss_fn(parsed_vars, opt_num, change_num):
    # years4f1 = Expected duration of function loss (in years)
    years4f1 = parsed_vars.get("years4f1")[opt_num][change_num]
    if years4f1 != 0:
        # qol_change4f1 = Average quality of life decrease during function loss (0 to 2)
        qol_change4f1 = parsed_vars.get("qol_change4f1")[opt_num][change_num]
        # income4f1 = Yearly income (in $) of person who would have function loss
        income4f1 = parsed_vars.get("income4f1")[opt_num][change_num]
        # age4f1 = Age (in years) of person who would have function loss
        age4f1 = parsed_vars.get("age4f1")[opt_num][change_num]
        # potential_lost4f1 = Percent of full potential lost due to health issue
        potential_lost4f1 = parsed_vars.get("potential_lost4f1")[opt_num][change_num]
        work_years = 65 - age4f1
        if work_years <= 0:
            change_wt = years4f1 * qol_change4f1 / 54.0
        elif work_years >= years4f1:
            change_wt = (
                years4f1 * income4f1 * (1.0 - potential_lost4f1 / 100.0) / 4050000.0
                + years4f1 * qol_change4f1 / 54.0
            )
        else:
            change_wt = (
                work_years * income4f1 * (1.0 - potential_lost4f1 / 100.0) / 4050000.0
                + years4f1 * qol_change4f1 / 54.0
            )
    else:
        change_wt = 0.0
    return change_wt


def bring_human_notenough_fn(parsed_vars, opt_num, change_num):
    # n_hum5f1 = How many humans brought into world
    n_hum5f1 = parsed_vars.get("n_hum5f1")[opt_num][change_num]
    # qol5f1 = Average quality of life expected (0 to 1)
    qol5f1 = parsed_vars.get("qol5f1")[opt_num][change_num]
    # n_hum_qol_down5f1 = How many pre-existing humans will experience decrease in their quality of life per human brought in
    n_hum_qol_down5f1 = parsed_vars.get("n_hum_qol_down5f1")[opt_num][change_num]
    # qol_down_hum5f1 = Average quality of life decrease expected (0 to 1)
    qol_down_hum5f1 = parsed_vars.get("qol_down_hum5f1")[opt_num][change_num]
    # years_hum5f1 = Average number of years quality of life decrease expected for pre-existing humans
    years_hum5f1 = parsed_vars.get("years_hum5f1")[opt_num][change_num]
    # n_anim_qol_down5f1 = How many animals will experience decrease in their quality of life per human brought in
    n_anim_qol_down5f1 = parsed_vars.get("n_anim_qol_down5f1")[opt_num][change_num]
    # qol_down_anim5f1 = Average quality of life decrease expected (0 to 1)
    qol_down_anim5f1 = parsed_vars.get("qol_down_anim5f1")[opt_num][change_num]
    # years_anim5f1 = Average number of years quality of life decrease expected for pre-existing animals
    years_anim5f1 = parsed_vars.get("years_anim5f1")[opt_num][change_num]
    # n_anim_die5f1 = How many animals will die due to resources taken per human brought in
    n_anim_die5f1 = parsed_vars.get("n_anim_die5f1")[opt_num][change_num]
    # years_more_anim5f1 = Average number of years longer animals otherwise would've lived
    years_more_anim5f1 = parsed_vars.get("years_more_anim5f1")[opt_num][change_num]
    # Assume animal quality of life of 0.6
    # n_tree_die5f1 = How many trees will die due to resources taken per human brought in
    n_tree_die5f1 = parsed_vars.get("n_tree_die5f1")[opt_num][change_num]
    # years_more_tree5f1 = Average number of years longer trees otherwise would've lived
    years_more_tree5f1 = parsed_vars.get("years_more_tree5f1")[opt_num][change_num]
    change_wt = n_hum5f1 * (
        n_hum_qol_down5f1 * years_hum5f1 * qol_down_hum5f1 / 54
        + n_anim_qol_down5f1 * years_anim5f1 * qol_down_anim5f1 / 540
        + n_anim_die5f1 * years_more_anim5f1 * 0.6 / 540
        + n_tree_die5f1 * years_more_tree5f1 / 54000
        - qol5f1 * 78 / 54
    )
    return change_wt


def bring_animal_notenough_fn(parsed_vars, opt_num, change_num):
    # n_anim5f2 = How many animals brought into world
    n_anim5f2 = parsed_vars.get("n_anim5f2")[opt_num][change_num]
    # qol5f2 = Average quality of life expected (0 to 1)
    qol5f2 = parsed_vars.get("qol5f2")[opt_num][change_num]
    # n_anim_qol_down5f2 = How many pre-existing animals will experience decrease in their quality of life per animal brought in
    n_anim_qol_down5f2 = parsed_vars.get("n_anim_qol_down5f2")[opt_num][change_num]
    # qol_down_anim5f2 = Average quality of life decrease expected (0 to 1)
    qol_down_anim5f2 = parsed_vars.get("qol_down_anim5f2")[opt_num][change_num]
    # years_anim5f2 = Average number of years quality of life decrease expected for pre-existing animals
    years_anim5f2 = parsed_vars.get("years_anim5f2")[opt_num][change_num]
    # n_anim_die5f2 = How many animals will die due to resources taken per animal brought in
    n_anim_die5f2 = parsed_vars.get("n_anim_die5f2")[opt_num][change_num]
    # years_more_anim5f2 = Average number of years longer animals otherwise would've lived
    years_more_anim5f2 = parsed_vars.get("years_more_anim5f2")[opt_num][change_num]
    # Assume animal quality of life of 0.6
    # n_plant_die5f2 = How many plants will die due to resources taken per animal brought in
    n_plant_die5f2 = parsed_vars.get("n_plant_die5f2")[opt_num][change_num]
    # years_more_plant5f2 = Average number of years longer plants otherwise would've lived
    years_more_plant5f2 = parsed_vars.get("years_more_plant5f2")[opt_num][change_num]
    change_wt = n_anim5f2 * (
        n_anim_qol_down5f2 * qol_down_anim5f2 * years_anim5f2 / 540
        + n_anim_die5f2 * years_more_anim5f2 * 0.6 / 540
        + n_plant_die5f2 * years_more_plant5f2 / 540000
        - qol5f2 * 3 / 540
    )
    return change_wt


def threat_pain_fn(parsed_vars, opt_num, change_num):
    # print(pain_levels)
    # tpain_level8f1 = Threatened physical pain level (0 to 10)
    tpain_level8f1 = parsed_vars.get("tpain_level8f1")[opt_num][change_num]
    if tpain_level8f1 != 0:
        # tduration8f1 = Threatened duration of physical pain (in minutes)
        tduration8f1 = parsed_vars.get("tduration8f1")[opt_num][change_num]
        # tlikelihood8f1 = Perceived percent likelihood that person would make good on the threat
        tlikelihood8f1 = parsed_vars.get("tlikelihood8f1")[opt_num][change_num]
        # unbuilt_value8f1 = Net value could build (in $) without unwanted pain
        unbuilt_value8f1 = parsed_vars.get("unbuilt_value8f1")[opt_num][change_num]
        if tpain_level8f1 > 7:
            change_wt_pain = (
                0.5
                * (
                    (1.903 * 10**-6) * (tduration8f1 * 100 ** (tpain_level8f1 - 7))
                    + unbuilt_value8f1 / 4050000.0
                )
                * tlikelihood8f1
                / 100.0
            )
        else:
            change_wt_pain = (
                0.5
                * (
                    (1.903 * 10**-13) * (tduration8f1 * 10**tpain_level8f1)
                    + unbuilt_value8f1 / 4050000.0
                )
                * tlikelihood8f1
                / 100.0
            )
    else:
        change_wt_pain = 0.0
    # tepain_level8f1 = Threatened emotional pain level (0 to 10)
    tepain_level8f1 = parsed_vars.get("tepain_level8f1")[opt_num][change_num]
    if tepain_level8f1 != 0:
        # teduration8f1 = Threatened duration of emotional pain (in minutes)
        teduration8f1 = parsed_vars.get("teduration8f1")[opt_num][change_num]
        # telikelihood8f1 = Perceived percent likelihood that person would make good on the threat
        telikelihood8f1 = parsed_vars.get("telikelihood8f1")[opt_num][change_num]
        # unbuilt_evalue8f1 = Net value could build (in $) without unwanted pain
        unbuilt_evalue8f1 = parsed_vars.get("unbuilt_evalue8f1")[opt_num][change_num]
        if tepain_level8f1 > 7:
            change_wt_epain = (
                0.5
                * (
                    (1.903 * 10**-6) * (teduration8f1 * 100 ** (tepain_level8f1 - 7))
                    + unbuilt_evalue8f1 / 4050000.0
                )
                * telikelihood8f1
                / 100.0
            )
        else:
            change_wt_epain = (
                0.5
                * (
                    (1.903 * 10**-13) * (teduration8f1 * 10**tepain_level8f1)
                    + unbuilt_evalue8f1 / 4050000.0
                )
                * telikelihood8f1
                / 100.0
            )
    else:
        change_wt_epain = 0.0
    change_wt = change_wt_pain + change_wt_epain
    return change_wt


def threat_function_loss_fn(parsed_vars, opt_num, change_num):
    # years8f2 = Expected duration of function loss (in years)
    years8f2 = parsed_vars.get("years8f2")[opt_num][change_num]
    if years8f2 != 0:
        # qol_change8f2 = Average quality of life decrease during function loss (0 to 2)
        qol_change8f2 = parsed_vars.get("qol_change8f2")[opt_num][change_num]
        # income8f2 = Yearly income (in $) of person who would have function loss
        income8f2 = parsed_vars.get("income8f2")[opt_num][change_num]
        # age8f2 = Age (in years) of person who would have function loss
        age8f2 = parsed_vars.get("age8f2")[opt_num][change_num]
        # potential_lost8f2 = Percent of full potential lost due to health issue
        potential_lost8f2 = parsed_vars.get("potential_lost8f2")[opt_num][change_num]
        # tlikelihood8f2 = Perceived percent likelihood that person would make good on the threat to hurt
        tlikelihood8f2 = parsed_vars.get("tlikelihood8f2")[opt_num][change_num]
        work_years = 65 - age8f2
        if work_years <= 0:
            change_wt = 0.5 * years8f2 * qol_change8f2 / 54.0 * tlikelihood8f2 / 100.0
        elif work_years >= years8f2:
            change_wt = (
                0.5
                * (
                    years8f2 * income8f2 * (1.0 - potential_lost8f2 / 100.0) / 4050000.0
                    + years8f2 * qol_change8f2 / 54.0
                )
                * tlikelihood8f2
                / 100.0
            )
        else:
            change_wt = (
                0.5
                * (
                    work_years
                    * income8f2
                    * (1.0 - potential_lost8f2 / 100.0)
                    / 4050000.0
                    + years8f2 * qol_change8f2 / 54.0
                )
                * tlikelihood8f2
                / 100.0
            )
    else:
        change_wt = 0.0
    return change_wt


def threat_kill_fn(parsed_vars, opt_num, change_num):
    # defaults = Do you want to use default values for this person's life value (Y or N)?
    defaults = parsed_vars.get("defaults")
    if defaults == "Y":
        lif_val = 1.54586
        life_expect = 78.27912
    else:
        # age8f3 = Age (in years) of person who would die
        age8f3 = parsed_vars.get("age8f3")[opt_num][change_num]
        # qol8f3 = Average quality of remaining life (-1 to 1)
        qol8f3 = parsed_vars.get("qol8f3")[opt_num][change_num]
        # income8f3 = Yearly income (in $) of person who would die
        income8f3 = parsed_vars.get("income8f3")[opt_num][change_num]
        # n_interact8f3 = Average number of people per year they would have interacted with if they lived
        n_interact8f3 = parsed_vars.get("n_interact8f3")[opt_num][change_num]
        # value_add8f3 = Average value add per person they would have interacted with
        value_add8f3 = parsed_vars.get("value_add8f3")[opt_num][change_num]
        # emot_depth8f3 = Average depth of relationships, for the emotional loss felt (0 to 1)
        emot_depth8f3 = parsed_vars.get("emot_depth8f3")[opt_num][change_num]
        # n_loss8f3 = How many relationships with people who would feel the emotional loss
        n_loss8f3 = parsed_vars.get("n_loss8f3")[opt_num][change_num]
        life_expect = (
            3.956 * 10**-5 * age8f3**3
            - 0.0023 * age8f3**2
            + 0.0767 * age8f3
            + 76.98
        )
        if age8f3 < 66:
            lif_val = (
                (65 - age8f3) * income8f3 / 4050000
                + (life_expect - age8f3) * (qol8f3 / 54 + n_interact8f3 * value_add8f3)
                + 0.01 * emot_depth8f3 * n_loss8f3
            )
        elif age8f3 < 71:
            lif_val = (life_expect - age8f3) * (
                qol8f3 / 54 + n_interact8f3 * value_add8f3
            ) + 0.01 * emot_depth8f3 * n_loss8f3
        else:
            lif_val = (life_expect - age8f3) * (
                qol8f3 / 54 + n_interact8f3 * value_add8f3
            ) + 0.001 * emot_depth8f3 * n_loss8f3
    # tlikelihood8f3 = Perceived percent likelihood that person would make good on the threat to kill
    tlikelihood8f3 = parsed_vars.get("tlikelihood8f3")[opt_num][change_num]
    change_wt = 0.5 * lif_val * tlikelihood8f3 / 100
    return change_wt


def emotional_pain_fn(parsed_vars, opt_num, change_num):
    # print(emotional_pain_levels)
    # epain_level10f1 = Average emotional pain level (1 to 10)
    epain_level10f1 = parsed_vars.get("epain_level10f1")[opt_num][change_num]
    # minutes10f1 = Duration of emotional pain (in minutes)
    minutes10f1 = parsed_vars.get("minutes10f1")[opt_num][change_num]
    # unbuilt_value10f1 = Net value could've been built (in $) if not for unwanted pain
    unbuilt_value10f1 = parsed_vars.get("unbuilt_value10f1")[opt_num][change_num]
    change_wt = (1.903 * 10**-13) * (
        minutes10f1 * 10**epain_level10f1
    ) + unbuilt_value10f1 / 4050000
    return change_wt


def steal_and_prop_change_fn(parsed_vars, opt_num, change_num):
    # cost13f1 = Replacement cost in dollars
    cost13f1 = parsed_vars.get("cost13f1")[opt_num][change_num]
    if cost13f1 != 0.0:
        # survive_change13f1 = Percent by which the property loss decreases survival likelihood over the next year
        survive_change13f1 = parsed_vars.get("survive_change13f1")[opt_num][change_num]
        # wt_death13f1 = Value weight of person(s) dying in the next year
        wt_death13f1 = parsed_vars.get("wt_death13f1")[opt_num][change_num]
        # eloss13f1 = Weight of emotional loss if property had special meaning
        eloss13f1 = parsed_vars.get("eloss13f1")[opt_num][change_num]
        # lessbuild13f1 = Likely decreased future value building (in $) due to property loss, over next year
        lessbuild13f1 = parsed_vars.get("lessbuild13f1")[opt_num][change_num]
        change_wt = (
            cost13f1 / 4050000.0
            + survive_change13f1 / 100 * wt_death13f1
            + eloss13f1
            + lessbuild13f1 / 4050000.0
        )
    else:
        change_wt = 0.0
    return change_wt


def property_repair_fn(parsed_vars, opt_num, change_num):
    # cost14f1 = Increase in the replacement cost in dollars due to the property repair
    cost14f1 = parsed_vars.get("cost14f1")[opt_num][change_num]
    # survive_change14f1 = Percent increases survival likelihood over the next year
    survive_change14f1 = parsed_vars.get("survive_change14f1")[opt_num][change_num]
    # wt_death14f1 = Value weight of person(s) dying in the next year
    wt_death14f1 = parsed_vars.get("wt_death14f1")[opt_num][change_num]
    # egain14f1 = Weight of emotional gain if property had special meaning
    egain14f1 = parsed_vars.get("egain14f1")[opt_num][change_num]
    # morebuild14f1 = Likely increased future value building (in $) due to property regain, over next year
    morebuild14f1 = parsed_vars.get("morebuild14f1")[opt_num][change_num]
    change_wt = (
        cost14f1 / 4050000.0
        + survive_change14f1 / 100 * wt_death14f1
        + egain14f1
        + morebuild14f1 / 4050000.0
    )
    return change_wt


def return_stolen_fn(parsed_vars, opt_num, change_num):
    # cost15f1= Replacement cost in dollars
    cost15f1 = parsed_vars.get("cost15f1")[opt_num][change_num]
    # survive_change15f1= Percent increases survival likelihood over the next year
    survive_change15f1 = parsed_vars.get("survive_change15f1")[opt_num][change_num]
    # wt_death15f1= Value weight of person(s) dying in the next year
    wt_death15f1 = parsed_vars.get("wt_death15f1")[opt_num][change_num]
    # egain15f1= Weight of emotional gain if property had special meaning
    egain15f1 = parsed_vars.get("egain15f1")[opt_num][change_num]
    # more_build15f1= Likely increased future value building due to property regain, over next year
    more_build15f1 = parsed_vars.get("more_build15f1")[opt_num][change_num]
    change_wt = (
        cost15f1 / 4050000.0
        + survive_change15f1 / 100 * wt_death15f1
        + egain15f1
        + more_build15f1
    )
    return change_wt


def masochistic_pain_fn(parsed_vars, opt_num, change_num):
    # print(pain_levels)
    # pain_level16f1 = Physical pain level (1 to 10)
    pain_level16f1 = parsed_vars.get("pain_level16f1")[opt_num][change_num]
    # minutes16f1 = Duration of physical pain (in minutes)
    minutes16f1 = parsed_vars.get("minutes16f1")[opt_num][change_num]
    # unbuilt_value16f1 = Net value could've been built (in $) if not for unwanted pain
    unbuilt_value16f1 = parsed_vars.get("unbuilt_value16f1")[opt_num][change_num]
    # For not taking responsibility for emotions
    # print(self_esteem_levels)
    # esteem_level16f1 = Starting self-esteem level (1 to 5)
    esteem_level16f1 = parsed_vars.get("esteem_level16f1")[opt_num][change_num]
    if esteem_level16f1 == 1:
        change_esteem = 0.0001
    elif esteem_level16f1 == 2:
        change_esteem = 0.005
    elif esteem_level16f1 == 3:
        change_esteem = 0.05
    elif esteem_level16f1 == 4:
        change_esteem = 0.5
    elif esteem_level16f1 == 5:
        change_esteem = 5
    change_wt = (
        (1.903 * 10**-13) * (minutes16f1 * 10**pain_level16f1)
        + unbuilt_value16f1
        + change_esteem
    )
    return change_wt


def sadistic_pleasure_fn(parsed_vars, opt_num, change_num):
    # breach_level17f1 = Level of ethical breach of the sadistic pleasure (1 to 5)
    breach_level17f1 = parsed_vars.get("breach_level17f1")[opt_num][change_num]
    # n_breach17f1 = Number of previous breaches at this level
    n_breach17f1 = parsed_vars.get("n_breach17f1")[opt_num][change_num]
    if breach_level17f1 == 1:
        change_wtb = 0.0001 / (n_breach17f1 + 1) ** 1.1
    elif breach_level17f1 == 2:
        change_wtb = 0.005 / (n_breach17f1 + 1) ** 1.1
    elif breach_level17f1 == 3:
        change_wtb = 0.05 / (n_breach17f1 + 1) ** 1.1
    elif breach_level17f1 == 4:
        change_wtb = 0.5 / (n_breach17f1 + 1) ** 1.1
    else:
        change_wtb = 5.0 / (n_breach17f1 + 1) ** 1.1
    # For not taking responsibility for emotions
    # print(self_esteem_levels)
    # esteem_level17f1 = Starting self-esteem level (1 to 5)
    esteem_level17f1 = parsed_vars.get("esteem_level17f1")[opt_num][change_num]
    if esteem_level17f1 == 1:
        change_wte = 0.0001
    elif esteem_level17f1 == 2:
        change_wte = 0.005
    elif esteem_level17f1 == 3:
        change_wte = 0.05
    elif esteem_level17f1 == 4:
        change_wte = 0.5
    elif esteem_level17f1 == 5:
        change_wte = 5
    change_wt = change_wte + change_wtb
    return change_wt


def conscience_fn(parsed_vars, opt_num, change_num):
    # breach_level18f1 = Level of ethical breach (1 to 6)
    breach_level18f1 = parsed_vars.get("breach_level18f1")[opt_num][change_num]
    # num_prev_breach18f1 = Number of previous breaches at this level, for something similar
    num_prev_breach18f1 = parsed_vars.get("num_prev_breach18f1")[opt_num][change_num]
    # Assume sum of value change weights for repeated breaches is bounded (exponent of 1.1 is > 1)
    if breach_level18f1 == 1:
        change_wt = 0.0001 / (num_prev_breach18f1 + 1) ** 1.1
    elif breach_level18f1 == 2:
        change_wt = 0.005 / (num_prev_breach18f1 + 1) ** 1.1
    elif breach_level18f1 == 3:
        change_wt = 0.05 / (num_prev_breach18f1 + 1) ** 1.1
    elif breach_level18f1 == 4:
        change_wt = 0.5 / (num_prev_breach18f1 + 1) ** 1.1
    elif breach_level18f1 == 5:
        change_wt = 2.0 / (num_prev_breach18f1 + 1) ** 1.1
    else:
        change_wt = 5.0 / (num_prev_breach18f1 + 1) ** 1.1
    return change_wt


def not_responsible_fn(parsed_vars, opt_num, change_num):
    # emo_notaccept19f1 = Percent of responsibility not accepting for emotions
    emo_notaccept19f1 = parsed_vars.get("emo_notaccept19f1")[opt_num][change_num]
    # act_notaccept19f1 = Percent of responsibility not accepting for actions
    act_notaccept19f1 = parsed_vars.get("act_notaccept19f1")[opt_num][change_num]
    # print(self_esteem_levels)
    # esteem_level19f1 = Starting self-esteem level (1 to 5)
    esteem_level19f1 = parsed_vars.get("esteem_level19f1")[opt_num][change_num]
    if esteem_level19f1 == 1:
        change_wt = 0.0001 * emo_notaccept19f1 / 100 + 0.0005 * act_notaccept19f1 / 100
    elif esteem_level19f1 == 2:
        change_wt = 0.005 * emo_notaccept19f1 / 100 + 0.025 * act_notaccept19f1 / 100
    elif esteem_level19f1 == 3:
        change_wt = 0.05 * emo_notaccept19f1 / 100 + 0.25 * act_notaccept19f1 / 100
    elif esteem_level19f1 == 4:
        change_wt = 0.5 * emo_notaccept19f1 / 100 + 2.5 * act_notaccept19f1 / 100
    elif esteem_level19f1 == 5:
        change_wt = 5 * emo_notaccept19f1 / 100 + 25 * act_notaccept19f1 / 100
    return change_wt


def be_responsible_fn(parsed_vars, opt_num, change_num):
    # emo_accept20f1 = Percent of responsibility accepting for emotions
    emo_accept20f1 = parsed_vars.get("emo_accept20f1")[opt_num][change_num]
    # act_accept20f1 = Percent of responsibility accepting for actions
    act_accept20f1 = parsed_vars.get("act_accept20f1")[opt_num][change_num]
    # print(self_esteem_levels)
    # esteem_level20f1 = Starting self-esteem level (1 to 5)
    esteem_level20f1 = parsed_vars.get("esteem_level20f1")[opt_num][change_num]
    if esteem_level20f1 == 1:
        change_wt = 0.0001 * emo_accept20f1 + 0.0005 * act_accept20f1
    elif esteem_level20f1 == 2:
        change_wt = 0.005 * emo_accept20f1 + 0.025 * act_accept20f1
    elif esteem_level20f1 == 3:
        change_wt = 0.05 * emo_accept20f1 + 0.25 * act_accept20f1
    elif esteem_level20f1 == 4:
        change_wt = 0.5 * emo_accept20f1 + 2.5 * act_accept20f1
    elif esteem_level20f1 == 5:
        change_wt = 5 * emo_accept20f1 + 25 * act_accept20f1
    return change_wt


def prethought_fn(parsed_vars, opt_num, change_num):
    # breach_down21f1 = Percent lowered likelihood of breach of ethics due to pre-thought and/or practicing critical thinking
    breach_down21f1 = parsed_vars.get("breach_down21f1")[opt_num][change_num]
    # print(self_esteem_levels)
    # esteem_level21f1 = Starting self-esteem level (1 to 5)
    esteem_level21f1 = parsed_vars.get("esteem_level21f1")[opt_num][change_num]
    if esteem_level21f1 == 1:
        change_wt = 0.0004 * breach_down21f1 / 100
    elif esteem_level21f1 == 2:
        change_wt = 0.02 * breach_down21f1 / 100
    elif esteem_level21f1 == 3:
        change_wt = 0.2 * breach_down21f1 / 100
    elif esteem_level21f1 == 4:
        change_wt = 2.0 * breach_down21f1 / 100
    elif esteem_level21f1 == 5:
        change_wt = 20 * breach_down21f1 / 100
    return change_wt


def animal_dying_fn(parsed_vars, opt_num, change_num):
    # anim_type24f1 = Is the animal a "nuisance" animal such as a musquito, tick, or harmful invasive species (Y or N)?
    anim_type24f1 = parsed_vars.get("anim_type24f1")[opt_num][change_num]
    # n_anim_die24f1 = How many equivalent animals dying
    n_anim_die24f1 = parsed_vars.get("n_anim_die24f1")[opt_num][change_num]
    # n_male24f1 = Number that are male
    n_male24f1 = parsed_vars.get("n_male24f1")[opt_num][change_num]
    n_female24f1 = n_anim_die24f1 - n_male24f1
    # years24f1 = Expected life remaining of animal (years)
    years24f1 = parsed_vars.get("years24f1")[opt_num][change_num]
    # qol24f1 = Assumed average remaining quality of life of animal (-1 to 1)
    qol24f1 = parsed_vars.get("qol24f1")[opt_num][change_num]
    # dollars_loss24f1 = Weight of value loss to humans (emotional, beauty, in equivalent $) from each animal
    dollars_loss24f1 = parsed_vars.get("dollars_loss24f1")[opt_num][change_num]
    # dollars_gain24f1 = Weight of human value gain or loss (in $) from each animal (from food, products, labor, etc.)
    dollars_gain24f1 = parsed_vars.get("dollars_gain24f1")[opt_num][change_num]
    if anim_type24f1 == "Y":
        change_wt = 0.0
    else:
        change_wt = (
            years24f1 * qol24f1 / 540 + dollars_loss24f1 - dollars_gain24f1 / 4050000
        ) * n_anim_die24f1
    return change_wt


def animal_pain_fn(parsed_vars, opt_num, change_num):
    # print(pain_levels)
    # pain25f1 = Assumed physical pain level for animal (0 to 10)
    pain25f1 = parsed_vars.get("pain25f1")[opt_num][change_num]
    # minutes25f1 = Duration of physical pain for animal (in minutes)
    minutes25f1 = parsed_vars.get("minutes25f1")[opt_num][change_num]
    change_wt = (1.903 * 10**-14) * minutes25f1 * 10**pain25f1
    return change_wt


def plant_dying_fn(parsed_vars, opt_num, change_num):
    # plant_type36f1 = Is the plant a "nuisance" plant such as a harmful invasive species (Y or N)
    plant_type36f1 = parsed_vars.get("plant_type36f1")[opt_num][change_num]
    # n_plant_die36f1 = How many equivalent plants dying
    n_plant_die36f1 = parsed_vars.get("n_plant_die36f1")[opt_num][change_num]
    # years36f1 = Expected life remaining of plant (years)
    years36f1 = parsed_vars.get("years36f1")[opt_num][change_num]
    # tree_or_plant36f1 = Tree or plant (T or P)?
    tree_or_plant36f1 = parsed_vars.get("tree_or_plant36f1")[opt_num][change_num]
    # wt_loss36f1 = Weight of value loss to humans (emotional, financial loss, beauty) from each plant
    wt_loss36f1 = parsed_vars.get("wt_loss36f1")[opt_num][change_num]
    # wt_gain36f1 = Weight of human value gain (financial) from each plant dying/being killed
    wt_gain36f1 = parsed_vars.get("wt_gain36f1")[opt_num][change_num]
    if plant_type36f1 == "Y":
        change_wt = 0.0
    elif tree_or_plant36f1 == "T":
        change_wt = (years36f1 / 54000 + wt_loss36f1 - wt_gain36f1) * n_plant_die36f1
    else:
        change_wt = (years36f1 / 540000 + wt_loss36f1 - wt_gain36f1) * n_plant_die36f1
    return change_wt


def pleasure_fn(parsed_vars, opt_num, change_num):
    # pleasure_level50f1 = Pleasure level (1 to 10)
    pleasure_level50f1 = parsed_vars.get("pleasure_level50f1")[opt_num][change_num]
    # minutes50f1 = Duration of pleasure (in minutes)
    minutes50f1 = parsed_vars.get("minutes50f1")[opt_num][change_num]
    change_wt = (1.903 * 10**-17) * (minutes50f1 * 10**pleasure_level50f1)
    return change_wt


def life_rights_fn(
    parsed_vars,
    villain,
    villain_prop,
    percent_culp,
    harms,
    num_pers,
    opt_num,
    change_num,
):
    # viol_type53f1 = Would this be an actual violation (A) of right to life or just a threatened (T) one (A or T)?
    viol_type53f1 = parsed_vars.get("viol_type53f1")[opt_num][change_num]
    if viol_type53f1 != "A":
        # tlikelihood53f1 = Perceived percent likelihood that the person making the threat would go through with it
        tlikelihood53f1 = parsed_vars.get("tlikelihood53f1")[opt_num][change_num]
    else:  # Default is to assume it is an actual violation of right to life
        tlikelihood53f1 = 100.0
    # worth_less53f1 = Has the person made clear that they want their life to be worth less than those of others involved (Y or N)?
    worth_less53f1 = parsed_vars.get("worth_less53f1")[opt_num][change_num]
    if worth_less53f1 == "Y":
        # If person purposely put themselves in harm's way to commit suicide, they effectively want their life valued at zero
        # want_valued53f1 = What fraction they want their right to life valued at with respect to others? (0=not at all, 1= equally)
        want_valued53f1 = parsed_vars.get("want_valued53f1")[opt_num][change_num]
    else:
        want_valued53f1 = 1.0
    if num_pers == 0:
        # num_viol53f1 = How many people's (including this one's) right to life would be violated at the same level of culpability?
        num_viol53f1 = parsed_vars.get("num_viol53f1")[opt_num][change_num]
    else:
        num_viol53f1 = num_pers
    if villain == 1:
        # If they put someone's life or body integrity in harm's way, they're considered to have 1/10,000th of rights of someone 100% in harm's way
        harms = 100.0
        percent_culp = max(percent_culp, 99.99)
    elif villain_prop == 1:
        # If only put someone's property in harm's way, they're considered to have rights of someone 100% in harm's way
        harms = 100.0
        percent_culp = max(percent_culp, 0.0)
    # People 100% harm's way have right to life weight of 100, those 0% in harm's way have weight of 1,000,000
    top_change_wt = (
        num_viol53f1
        * want_valued53f1
        * (tlikelihood53f1 / 100)
        * 1000000.0
        * (1 - (harms / 100) * (percent_culp / 100))
        / (1 + 9999 * (harms / 100))
    )
    return top_change_wt


def body_rights_fn(
    parsed_vars,
    villain,
    villain_prop,
    percent_culp,
    harms,
    num_pers,
    opt_num,
    change_num,
):
    # viol_type54f1 = Would this be an actual violation (A) of right to body integrity or just a threatened (T) one (A or T)?
    viol_type54f1 = parsed_vars.get("viol_type54f1")[opt_num][change_num]
    if viol_type54f1 != "A":
        # tlikelihood54f1 = Perceived percent likelihood that person would make good on the threat
        tlikelihood54f1 = parsed_vars.get("tlikelihood54f1")[opt_num][change_num]
    else:
        tlikelihood54f1 = 100.0
    if num_pers == 0:
        # num_viol54f1 = How many people's (including this one's) right to body integrity would be violated to the same level, including culpability
        num_viol54f1 = parsed_vars.get("num_viol54f1")[opt_num][change_num]
    else:
        num_viol54f1 = num_pers
    if num_viol54f1 != 0:
        # Note that the value destructions of both the rights violation and the act itself will be counted
        # For unwanted physical pain:
        pain = unwanted_pain_fn(parsed_vars, opt_num, change_num)
        # For loss of function due to health issues:
        health = function_loss_fn(parsed_vars, opt_num, change_num)
        # For threat of physical violence or emotional pain:
        tpain = threat_pain_fn(parsed_vars, opt_num, change_num)
        # For threat of physical violence to the point of a long-term health issue:
        thealth = threat_function_loss_fn(parsed_vars, opt_num, change_num)
        # For threat of physical violence to the point of death:
        tkill = threat_kill_fn(parsed_vars, opt_num, change_num)
        # For physical restraint:
        restraint = restraint_fn(parsed_vars, opt_num, change_num)
        # worth_less54f1 = Has the person made clear that they want their body integrity to be worth less than those of others involved (Y or N)?
        worth_less54f1 = parsed_vars.get("worth_less54f1")[opt_num][change_num]
        if worth_less54f1 == "Y":
            # want_valued54f1 = How much do they want their right to body integrity valued with respect to others? (0=not at all, 1=equally)
            want_valued54f1 = parsed_vars.get("want_valued54f1")[opt_num][change_num]
        else:
            want_valued54f1 = 1.0
        villain_fact = 1.0
        if villain == 1:
            # If they put someone's life or body integrity in harm's way, they're considered to have 1/10,000th of rights of someone 100% in harm's way
            harms = 100.0
            percent_culp = max(percent_culp, 99.99)
            villain_fact = 0.0001
        elif villain_prop == 1:
            # If they only put someone's property in harm's way, they're considered to have rights of someone 100% in harm's way
            harms = 100.0
            percent_culp = max(percent_culp, 0.0)
        top_change_wt = (
            num_viol54f1
            * want_valued54f1
            * (tlikelihood54f1 / 100)
            * (
                (
                    1000000.0
                    * (1 - (harms / 100) * (percent_culp / 100))
                    * (pain + health + tpain + thealth + tkill + restraint)
                )
                / (1 + 9999 * (harms / 100))
                + villain_fact * (pain + health + tpain + thealth + tkill + restraint)
            )
        )
        rights_change_wt = (
            num_viol54f1
            * want_valued54f1
            * (tlikelihood54f1 / 100)
            * (
                1000000.0
                * (1 - (harms / 100) * (percent_culp / 100))
                * (pain + health + tpain + thealth + tkill + restraint)
            )
            / (1 + 9999 * (harms / 100))
        )
    else:
        top_change_wt = 0.0
        rights_change_wt = 0.0
    return (
        top_change_wt,
        pain,
        health,
        tpain,
        thealth,
        tkill,
        restraint,
        rights_change_wt,
    )


def restraint_fn(parsed_vars, opt_num, change_num):
    # restraint_lev54f2 = What level of restraint (0 through 5)
    restraint_lev54f2 = parsed_vars.get("restraint_lev54f2")[opt_num][change_num]
    # Assume quality of life of 0.5
    if restraint_lev54f2 == 1:
        # time_exclude54f2 = How much time excluded (in minutes)
        time_exclude54f2 = parsed_vars.get("time_exclude54f2")[opt_num][change_num]
        change_wt = 0.1 * time_exclude54f2 / 60 / 24 / 365 / 54 * 0.5
    elif restraint_lev54f2 == 2:
        # time_imprison54f2 = How much time imprisoned (in minutes)
        time_imprison54f2 = parsed_vars.get("time_imprison54f2")[opt_num][change_num]
        change_wt = 0.5 * time_imprison54f2 / 60 / 24 / 365 / 54 * 0.5
    elif restraint_lev54f2 == 3:
        # time_back54f2 = How much time to get back to original location (in minutes)
        time_back54f2 = parsed_vars.get("time_back54f2")[opt_num][change_num]
        # money_back54f2 = How much money to get back to original location (in $)
        money_back54f2 = parsed_vars.get("money_back54f2")[opt_num][change_num]
        change_wt = (
            0.5 * time_back54f2 / 60 / 24 / 365 / 54 * 0.5 + money_back54f2 / 4050000
        )
    elif restraint_lev54f2 == 4:
        # time_restrain54f2 = How much time physically restrained (in minutes)
        time_restrain54f2 = parsed_vars.get("time_restrain54f2")[opt_num][change_num]
        change_wt = 1.0 * time_restrain54f2 / 60 / 24 / 365 / 54 * 0.5
    elif restraint_lev54f2 == 5:
        # time_unconscious54f2 = How much time unconscious (in minutes)
        time_unconscious54f2 = parsed_vars.get("time_unconscious54f2")[opt_num][
            change_num
        ]
        change_wt = 1.0 * time_unconscious54f2 / 60 / 24 / 365 / 54 * 0.5
    else:
        change_wt = 0.0
    return change_wt


def prop_rights_fn(
    parsed_vars,
    villain,
    villain_prop,
    culp_prop,
    harm_prop,
    prop_val,
    num_pers,
    opt_num,
    change_num,
):
    # viol_type55f1 = Would this be an actual violation (A) of right to property or just a threatened (T) one?
    viol_type55f1 = parsed_vars.get("viol_type55f1")[opt_num][change_num]
    if viol_type55f1 != "A":
        # tlikelihood55f1 = Perceived percent likelihood that person would make good on the threat
        tlikelihood55f1 = parsed_vars.get("tlikelihood55f1")[opt_num][change_num]
    else:
        tlikelihood55f1 = 100.0
    # For property damaged and/or stolen:
    if num_pers == 0:
        # num_viol55f1 = How many people's (including this one's) right to property would be violated at the same level of culpability?
        num_viol55f1 = parsed_vars.get("num_viol55f1")[opt_num][change_num]
        property_value = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
    else:
        num_viol55f1 = num_pers
        property_value = steal_and_prop_change_fn(parsed_vars, opt_num, change_num)
    if harm_prop > 0:
        culp_prop55f1 = culp_prop
    else:
        culp_prop55f1 = 0.0
    if villain == 1:
        # If they put someone's life or body integrity in harm's way, their rights drop to 1/10,000th the value for property in harm's way
        harm_prop = 100.0
        culp_prop55f1 = max(culp_prop55f1, 99.99)
    elif villain_prop == 1:
        # If they only put someone's property in harm's way, their rights drop to 1/5th the value for property in harm's way
        harm_prop = 100.0
        culp_prop55f1 = max(culp_prop55f1, 80.0)
    # Note that the value destructions of both the rights violation and the property loss will be counted
    top_change_wt = (
        num_viol55f1
        * (1 - harm_prop / 100 * culp_prop55f1 / 100)
        * (tlikelihood55f1 / 100)
        * ((10000.0 * property_value) / (1 + 99 * (harm_prop / 100)) + property_value)
    )
    # fix_intent55f1 = Is there intent of those responsible to repair this property value destruction later (Y/N)?
    fix_intent55f1 = parsed_vars.get("fix_intent55f1")[opt_num][change_num]
    if fix_intent55f1 == "Y":
        # likelihood_fix55f1 = The percent likelihood of this value destruction repair occurring
        likelihood_fix55f1 = parsed_vars.get("likelihood_fix55f1")[opt_num][change_num]
        # percent_fix55f1 = What percentage of the value destruction is likely to be repaired (up to max of 99)?
        percent_fix55f1 = parsed_vars.get("percent_fix55f1")[opt_num][change_num]
    else:
        likelihood_fix55f1 = 0.0
        percent_fix55f1 = 0.0
    top_change_wt_fixed = top_change_wt * (
        1 - likelihood_fix55f1 / 100.0 * percent_fix55f1 / 100.0
    )
    rights_change_wt = (
        num_viol55f1
        * (1 - harm_prop / 100 * culp_prop55f1 / 100)
        * (tlikelihood55f1 / 100)
        * (10000.0 * property_value)
        / (1 + 99 * (harm_prop / 100))
    ) * (1 - likelihood_fix55f1 / 100.0 * percent_fix55f1 / 100.0)
    return top_change_wt, property_value, top_change_wt_fixed, rights_change_wt

In [4]:
pain_levels = (
    "Pain levels: 0 = No pain, 1 = Hardly notice, 2 = Notice, doesn’t interfere with activities, "
    "3 = Sometimes distracts, 4 = Distracts, can do usual activities, 5 = Interrupts some activities, "
    "6 = Hard to ignore, avoid usual activities, 7 = Focus of attention, prevents doing daily activities, "
    "8 = Awful, hard to do anything, 9 = Can’t bear the pain, unable to do anything, "
    "10 = As bad as could be, nothing else matters"
)
self_esteem_levels = (
    "Self-esteem levels: 1 - effectively totally reactive, "
    "takes no responsibility for actions or emotions, "
    "2 - takes limited responsibility for actions, none for emotions, "
    "3 - takes some responsibility for actions, some for emotions, "
    "4 - takes responsibility for most actions and emotions, "
    "5 - takes full responsibility for actions and emotions, grateful for life and joyous"
)
ethical_breach_levels = (
    "Ethical breach levels: 1 - telling white lies, going against small values/ethics, "
    "destroying a small amount of value unnecessarily (such as wasting food/resources), "
    "lying, stealing, cheating when up against an 'adversary' who's threatening to do worse; "
    "2 - not helping others in significant need (who you don't see) when you could, "
    "not holding accountable someone who you're responsible for holding accountable, for a small breach; "
    "3 - lying, stealing, cheating, not keeping your word; 4 - being significantly violent, "
    "not helping someone in significant need (right in front of you) when you could, "
    "not holding accountable someone who you're responsible for holding accountable, for a large breach; "
    "5 - going against a large value/important ethic, such as not to murder; "
    "6 - extreme torturing"
)
emotional_pain_levels = (
    "Emotional pain levels: 0 = No pain, 1 = Hardly notice, 2 = Notice, doesn’t interfere with activities, "
    "3 = Sometimes distracts, 4 = Distracts, can do usual activities, 5 = Interrupts some activities, "
    "6 = Hard to ignore, avoid usual activities, 7 = Focus of attention, prevents doing daily activities, "
    "8 = Awful, hard to do anything, 9 = Can’t bear the pain, unable to do anything, "
    "10 = As bad as could be, nothing else matters"
)

value_change_output = "*****Value change weight is: "


def parse_value_file(file_path):
    try:
        with open(file_path, "r") as file:
            options_dict = {}
            for line in file:
                parts = line.strip().split(".")
                if len(parts) == 2:
                    number = int(parts[0].strip())
                    option = parts[1].strip()
                    options_dict[number] = option
    except FileNotFoundError:
        print(f'File "{file_path}" not found.')
    except Exception as e:
        print(f"An error occurred while trying to read the value change list: {str(e)}")
    n_options = len(options_dict)
    return n_options, options_dict

In [5]:
def parse_decision_file(content):
    input_vars = {}
    for line in content.strip().split("\n"):
        if line[0] != "#":
            key, value = line.strip().split("=")
            key = key.strip()
            value = value.strip()
            if value[0] != "'":
                try:
                    # Try to evaluate the value as a literal expression
                    value_non_str = ast.literal_eval(value)
                    input_vars[key] = value_non_str
                except (SyntaxError, ValueError):
                    if value.startswith("[") and value.endswith("]"):
                        # Check if the value is a list and parse it accordingly, including checking for nested lists
                        if "[" not in value[1:-1]:
                            value_list = [
                                item.strip() for item in value[1:-1].split(",")
                            ]
                        else:
                            value_list = []
                            item_temp = ""
                            for item in value[1:-1].split(","):
                                str_list = []
                                if "[" in item and "]" in item:
                                    try:
                                        value_non_str = ast.literal_eval(item)
                                        value_list.append(value_non_str)
                                    except (SyntaxError, ValueError):
                                        value_non_str = item[1:-1]
                                        str_list.append(value_non_str)
                                        value_list.append(str_list)
                                else:
                                    item_temp = item_temp + item + ","
                                    if "[" in item_temp and "]" in item_temp:
                                        item_temp = item_temp.strip(",")
                                        value_list1 = [
                                            item.strip()
                                            for item in item_temp[1:-1].split(",")
                                        ]
                                        value_list.append(value_list1)
                                        item_temp = ""
                        input_vars[key] = value_list
                    else:
                        # Keep it as a string if not a list
                        input_vars[key] = value
            else:
                input_vars[key] = value
    return input_vars


inputs_file = "/InputTrolley1.txt"
try:
    with open(inputs_file, "r") as file:
        file_content = file.read()
except FileNotFoundError:
    print(f'File "{inputs_file}" not found.')
parsed_vars = parse_decision_file(file_content)
print(parsed_vars)

{'n_opt': 2, 'options': ['Nothing', 'Turn'], 'prop_value': [0, 0], 'n_people': 2, 'life_value': [1.54586, 1.54586], 'defaults': 'Y', 'person_number': [1, 2], 'num_persons': [5, 1], 'n_harms': 1, 'villainy': [0, 0], 'villainy_prop': [0, 0], 'percent_culpability': [50, 50], 'harms_way': [100, 50], 'culp_prop': [0, 0], 'harms_way_prop': [0, 0], 'change_inputs': [{0: 2, 1: 18, 2: 53}, {0: 2, 1: 18, 2: 53}], 'person_change': [[1, 0, 1], [2, 0, 2]], 'change_waivers': [[0, 0, 0], [0, 0, 0]], 'likelihood_fix_destroy': [[0, 0, 0], [0, 0, 0]], 'percent_fix_destroy': [[0, 0, 0], [0, 0, 0]], 'n_die1': [{}, {}], 'n_pain1': [{}, {}], 'na_die1': [{}, {}], 'na_pain1': [{}, {}], 'np_die1': [{}, {}], 'n_notborn1': [{}, {}], 'na_notborn1': [{}, {}], 'np_notborn1': [{}, {}], 'time_prob1': [{}, {}], 'prob_i1': [{}, {}], 'prob_f1': [{}, {}], 'time_prob2': [{0: 1}, {0: 1}], 'prob_i2': [{0: 0}, {0: 0}], 'prob_f2': [{0: 100}, {0: 100}], 'n_die2': [{0: 1}, {0: 1}], 'waiver2': [{0: 1}, {0: 1}], 'time_prob3': [{}

In [6]:
# These are "proof-of-concept" calculations - the value weight parameters haven't yet been refined
# so don't trust the program outputs!!
change_rights_wt = []
options_calcs = []
rights_change = []

# n_opt = Integer number of decision options
n_opt = parsed_vars.get("n_opt", 0)
# options = List of decision option names
options = parsed_vars.get("options", "")
# prop_value = Value weight of property calculated from steal_and_prop_destroy_fn
prop_value = parsed_vars.get("prop_value", [])
# Value weight of lives set to default or calculated from life_value_fn
life_value = parsed_vars.get("life_value", [])
# person_number = The number assigned to a person (set of equivalent people) to keep track of them (1,2,3….n_people)
person_number = parsed_vars.get("person_number", [])
# num_persons = How many people are equivalent to this person, including them themselves?
num_persons = parsed_vars.get("num_persons", [])
# change_inputs = Value change numbers from the list in the calc_change function
change_inputs = parsed_vars.get("change_inputs", [])
# person_change = Which person number does this value change correspond to (enter 0 if none)?
person_change = parsed_vars.get("person_change", [])
# change_waivers = Percentages of involved people who'd likely want to waive consideration of each value
# change for them if they knew of a bigger corresponding value destruction?
change_waivers = parsed_vars.get("change_waivers", [])
# likelihood_fix_destroy = The percent likelihood of this value destruction repair occurring
likelihood_fix_destroy = parsed_vars.get("likelihood_fix_destroy", [])
# percent_fix_destroy = Percentage of the value destruction likely to be repaired
percent_fix_destroy = parsed_vars.get("percent_fix_destroy", [])

for ii in range(0, n_opt):
    print("")
    print(f"For decision option # {ii+1} ({options[ii]}):")
    if ii == 0:
        file_path = "/ValueChangeMinSet.txt"
        n_change, changes_dict = parse_value_file(file_path)
    top_change_tot_wt = 0.0
    rights_change_tot_wt = 0.0
    for i in range(len(change_inputs[ii])):
        print("")
        if change_inputs[ii][i] != 0:
            print(f"For: {changes_dict[change_inputs[ii][i]]}")
        if person_change[ii][i] != 0:
            jj = person_change[ii][i] - 1
            top_change_wt, rights_change_wt = calc_change(
                parsed_vars,
                change_inputs[ii][i],
                num_persons[jj],
                life_value[jj],
                change_waivers[ii][i],
                prop_value[jj],
                ii,
                i,
                jj + 1,
            )
        else:
            top_change_wt, rights_change_wt = calc_change(
                parsed_vars, change_inputs[ii][i], 0, 0, 0, 0, ii, i, 1
            )
        top_change_tot_wt += top_change_wt * (
            1 - (likelihood_fix_destroy[ii][i] / 100 * percent_fix_destroy[ii][i] / 100)
        )
        rights_change_tot_wt += rights_change_wt
    change_rights_wt.append(rights_change_tot_wt)
    # Store net value changes
    options_calcs.append(top_change_tot_wt)
    rights_change.append(change_rights_wt[ii])

print("")
print("")
# Identify and output the best option in terms of most value change/least value destruction
# and least rights violations
maximum = max(options_calcs)
min_rights_viol = max(rights_change)
max_val_list = [i + 1 for i, value in enumerate(options_calcs) if value == maximum]
min_rights_list = [
    i + 1 for i, value in enumerate(rights_change) if value == min_rights_viol
]
print(f"The most positive value change weight is calculated to be {maximum}.")
if len(max_val_list) == 1:
    print(
        f"*****The best overall value change option is calculated to be: #{max_val_list[0]} : {options[max_val_list[0]-1]}."
    )
elif len(max_val_list) != n_opt:
    print(
        f"*****The best overall value change options are calculated to be: #'s{max_val_list}."
    )
else:
    print(
        f"*****The best overall value change options are calculated to be all options."
    )
print("")
print(
    f"The least overall rights-violating weight is calculated to be {min_rights_viol}."
)
if len(min_rights_list) == 1:
    print(
        f"*****The least overall rights-violating option is calculated to be: #{min_rights_list[0]} : {options[min_rights_list[0]-1]}."
    )
elif len(min_rights_list) != n_opt:
    print(
        f"*****The least overall rights-violating options are calculated to be: #'s{min_rights_list}."
    )
else:
    print(
        f"*****The least overall rights-violating options are calculated to be all options."
    )
print("")
print(
    "*****The estimated net value builds (positive) or destructions (negative) for each option are:"
)
for i in range(n_opt):
    print(f"{options_calcs[i]} for {options[i]} (rights = {rights_change[i]})")


For decision option # 1 (Nothing):

For: Someone dying, or increasing/decreasing its probability
*****Value change weight is:  -7.7293

For: Going against one’s conscience, or increasing/decreasing its probability
*****Value change weight is:  -0.5

For: Violating right to life, or increasing/decreasing its probability
*****Value change weight is:  -250.0

For decision option # 2 (Turn):

For: Someone dying, or increasing/decreasing its probability
*****Value change weight is:  -1.54586

For: Going against one’s conscience, or increasing/decreasing its probability
*****Value change weight is:  -0.0001

For: Violating right to life, or increasing/decreasing its probability
*****Value change weight is:  -149.98500149985003


The most positive value change weight is calculated to be -151.53096149985004.
*****The best overall value change option is calculated to be: #2 : Turn.

The least overall rights-violating weight is calculated to be -149.98500149985003.
*****The least overall rights